In [ ]:
import random
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import numpy as np
import math
import random
import unidecode
import datetime
import time
from time import strptime
import math
from IPython.core.display import display, HTML
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)
display(HTML("<style>.container { width:80% !important; }</style>"))

# Below are dicts, lists, and strings that will be used in later functions
header_name = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
acronym_to_city_dict = {'ATL': 'Atlanta',
                        'WSH': 'Washington',
                        'NYM': 'New York Mets',
                        'PHI': 'Philadelphia',
                        'MIA': 'Miami',
                        'STL': 'St. Louis',
                        'MIL': 'Milwaukee',
                        'CHC': 'Chicago Cubs',
                        'CIN': 'Cincinnati',
                        'PIT': 'Pittsburgh',
                        'LAD': 'Los Angeles Dodgers',
                        'ARI': 'Arizona',
                        'SF': 'San Francisco',
                        'COL': 'Colorado',
                        'SD': 'San Diego',
                        'NYY': 'New York Yankees',
                        'TB': 'Tampa Bay',
                        'BOS': 'Boston',
                        'TOR': 'Toronto',
                        'BAL': 'Baltimore',
                        'MIN': 'Minnesota',
                        'CLE': 'Cleveland',
                        'CHW': 'Chicago White Sox',
                        'KC': 'Kansas City',
                        'DET': 'Detroit',
                        'HOU': 'Houston',
                        'OAK': 'Oakland',
                        'TEX': 'Texas',
                        'LAA': 'Los Angeles Angels',
                        'SEA': 'Seattle'}
#acronym_to_city_dict = {'NYY': 'New York Yankees', 'ATL': 'Atlanta'}
nickname_to_city_dict = {'braves': 'Atlanta',
                        'nationals': 'Washington',
                        'mets': 'New York Mets',
                        'phillies': 'Philadelphia',
                        'marlins': 'Miami',
                        'cardinals': 'St. Louis',
                        'brewers': 'Milwaukee',
                        'cubs': 'Chicago Cubs',
                        'reds': 'Cincinnati',
                        'pirates': 'Pittsburgh',
                        'dodgers': 'Los Angeles Dodgers',
                        'diamondbacks': 'Arizona',
                        'd-backs': 'Arizona',
                        'giants': 'San Francisco',
                        'rockies': 'Colorado',
                        'padres': 'San Diego',
                        'yankees': 'New York Yankees',
                        'rays': 'Tampa Bay',
                        'red sox': 'Boston',
                        'blue jays': 'Toronto',
                        'orioles': 'Baltimore',
                        'twins': 'Minnesota',
                        'indians': 'Cleveland',
                        'white sox': 'Chicago White Sox',
                        'royals': 'Kansas City',
                        'tigers': 'Detroit',
                        'astros': 'Houston',
                        'athletics': 'Oakland',
                        'rangers': 'Texas',
                        'angels': 'Los Angeles Angels',
                        'mariners': 'Seattle'}
league = {'Atlanta': 'NL',
          'Washington': 'NL',
          'New York Mets': 'NL',
          'Philadelphia': 'NL',
          'Miami': 'NL',
          'St. Louis': 'NL',
          'Milwaukee': 'NL',
          'Chicago Cubs': 'NL',
          'Cincinnati': 'NL',
          'Pittsburgh': 'NL',
          'Los Angeles Dodgers': 'NL',
          'Arizona': 'NL',
          'San Francisco': 'NL',
          'Colorado': 'NL',
          'San Diego': 'NL',
          'New York Yankees': 'AL',
          'Tampa Bay': 'AL',
          'Boston': 'AL',
          'Toronto': 'AL',
          'Baltimore': 'AL',
          'Minnesota': 'AL',
          'Cleveland': 'AL',
          'Chicago White Sox': 'AL',
          'Kansas City': 'AL',
          'Detroit': 'AL',
          'Houston': 'AL',
          'Oakland': 'AL',
          'Texas': 'AL',
          'Los Angeles Angels': 'AL',
          'Seattle': 'AL'}
team_num_dict = {'Atlanta': 16,
          'Washington': 24,
          'New York Mets': 25,
          'Philadelphia': 26,
          'Miami': 20,
          'St. Louis': 28,
          'Milwaukee': 23,
          'Chicago Cubs': 17,
          'Cincinnati': 18,
          'Pittsburgh': 27,
          'Los Angeles Dodgers': 22,
          'Arizona': 15,
          'San Francisco': 30,
          'Colorado': 19,
          'San Diego': 29,
          'New York Yankees': 9,
          'Tampa Bay': 12,
          'Boston': 3,
          'Toronto': 14,
          'Baltimore': 2,
          'Minnesota': 8,
          'Cleveland': 5,
          'Chicago White Sox': 4,
          'Kansas City': 7,
          'Detroit': 6,
          'Houston': 21,
          'Oakland': 10,
          'Texas': 13,
          'Los Angeles Angels': 1,
          'Seattle': 11}
mlb_teams = ['braves', 'nationals', 'mets', 'phillies', 'marlins', 
         'cardinals', 'brewers', 'cubs', 'reds', 'pirates', 
         'dodgers', 'd-backs', 'giants', 'rockies', 'padres', 
         'yankees', 'rays', 'red sox', 'blue jays', 'orioles', 'twins',
         'indians', 'white sox', 'royals', 'tigers', 'astros', 'athletics',
         'rangers', 'angels', 'mariners']

In [ ]:
class Batter_new:
    """Class that holds information for a Batter.
    
    Attributes:

    """
    def __init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, hbp_L, so_L, singles_R, 
                 doubles_R, triples_R, home_runs_R, walks_R, hbp_R, so_R,
                 singles_Home, doubles_Home, triples_Home, home_runs_Home, walks_Home, hbp_Home, so_Home, singles_Away, 
                 doubles_Away, triples_Away, home_runs_Away, walks_Away, hbp_Away, so_Away, sb_rate, sb_success):
        """Initializes values for this class
        
        Args:
            team: team name of batter
            name: batter name
            hand: Batting hand of hitter(L, R, or S)
            singles_L: fraction of plate appearances ending in a single vs. lefties
            doubles_L: fraction of plate appearances ending in a double vs. lefties
            triples_L: fraction of plate appearances ending in a triple vs. lefties
            home_runs_L: fraction of plate appearances ending in a home run vs. lefties
            walks_L: fraction of plate appearances ending in a walk vs. lefties
            hbp_L: fraction of plate appearances ending in a hbp vs. lefties
            so_L: fraction of plate appearances ending in a strikeout vs. lefties
            singles_RL: fraction of plate appearances ending in a single vs. righties
            doubles_R: fraction of plate appearances ending in a double vs. righties
            triples_R: fraction of plate appearances ending in a triple vs. righties
            home_runs_R: fraction of plate appearances ending in a home run vs. righties
            walks_R: fraction of plate appearances ending in a walk vs. righties
            hbp_R: fraction of plate appearances ending in a hbp vs. righties
            so_R: fraction of plate appearances ending in a strikeout vs. righties
            sb_rate: approximate percentage of time player will steal base given opportunity
            sb_success: fraction of stolen base attempts ending in success
            bsr: fangraphs stat representing projected runs created by a base runner. Higher number is better
            fld: bsr: fangraphs stat representing projected runs created by a fielder. higher number is better
        """
        self.team = team
        self.name = name
        self.hand = hand
        self.singles_L = singles_L
        self.doubles_L = doubles_L
        self.triples_L = triples_L
        self.home_runs_L = home_runs_L
        self.walks_L = walks_L
        self.hbp_L = hbp_L
        self.so_L = so_L
        self.slg_L = walks_L+singles_L+2*doubles_L+3*triples_L+4*home_runs_L
        self.singles_R = singles_R
        self.doubles_R = doubles_R
        self.triples_R = triples_R
        self.home_runs_R = home_runs_R
        self.walks_R = walks_R
        self.hbp_R = hbp_R
        self.so_R = so_R
        self.slg_R = walks_R+singles_R+2*doubles_R+3*triples_R+4*home_runs_R
        self.singles_Home = singles_Home
        self.doubles_Home = doubles_Home
        self.triples_Home = triples_Home
        self.home_runs_Home = home_runs_Home
        self.walks_Home = walks_Home
        self.hbp_Home = hbp_Home
        self.so_Home = so_Home
        self.slg_Home = walks_Home+singles_Home+2*doubles_Home+3*triples_Home+4*home_runs_Home
        self.singles_Away = singles_Away
        self.doubles_Away = doubles_Away
        self.triples_Away = triples_Away
        self.home_runs_Away = home_runs_Away
        self.walks_Away = walks_Away
        self.hbp_Away = hbp_Away
        self.so_Away = so_Away
        self.slg_Away = walks_Away+singles_Away+2*doubles_Away+3*triples_Away+4*home_runs_Away
        self.sb_rate = sb_rate
        self.sb_success = sb_success 
        
        #self.cum values represent cumulative (season) totals for a batter
        self.cum_singles_L = 0
        self.cum_doubles_L = 0
        self.cum_triples_L = 0
        self.cum_home_runs_L = 0
        self.cum_walks_L = 0
        self.cum_hbp_L = 0
        self.cum_so_L = 0
        self.cum_pa_L = 0
        self.cum_singles_R = 0
        self.cum_doubles_R = 0
        self.cum_triples_R = 0
        self.cum_home_runs_R = 0
        self.cum_walks_R = 0
        self.cum_hbp_R = 0
        self.cum_so_R = 0
        self.cum_pa_R = 0
        self.cum_runs = 0
        self.cum_rbi = 0
        self.cum_sb = 0
        self.cum_cs = 0
        self.cum_sf = 0
        self.cum_sh = 0
        self.cum_gdp = 0
        self.cum_G = 0
        self.cum_GS = 0
     
    def hits(self):
        """cumulative hits of a batter
        
        Returns:
            int: number of cumulative hits
        """
        return(self.cum_singles_L+self.cum_doubles_L+self.cum_triples_L+self.cum_home_runs_L+
               self.cum_singles_R+self.cum_doubles_R+self.cum_triples_R+self.cum_home_runs_R)
    
    def avg(self):
        """Calculates avg of a player
        
        Returns:
            float: batting average in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf+self.cum_sh)>0):
            return(round(float((self.hits())/((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf))), 3))
        else:
            return 0    
        
    def obp(self):
        """Calculates on base percentage of a player
        
        Returns:
            float: opb in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)>0):
            return(round((self.hits()+(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R))/(self.cum_pa_L+self.cum_pa_R), 3))
        else:
            return 0
        
    def slg(self):
        """Calculates slugging percentage of a player
        
        Returns:
            float: slg in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf)>0):
            return(round(((self.cum_singles_L+self.cum_singles_R)+2*(self.cum_doubles_L+self.cum_doubles_R)+
                           3*(self.cum_triples_L+self.cum_triples_R)+4*(self.cum_home_runs_L+self.cum_home_runs_R))/
                           ((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf+self.cum_sh)), 3))
        else:
            return 0
        
    def ops(self):
        """Calculates on base plus slugging of a player
        
        Returns:
            float: ops in decimal form
            """
        return(self.slg() + self.obp())
    
    def strikeouts(self):
        """Number of strikeouts of a batter
        
        Returns:
            float: total strikeouts
        """
        return(self.cum_so_L+self.cum_so_R)
    
    def print_stats(self):
        """Prints some stats of a player
        """
        print("Avg: " + str(round(self.avg(), 3)))
        print("OBP: " + str(round(self.obp(), 3)))
        print("SLG: " + str(round(self.slg(), 3)))
        print("OPS: " + str(round(self.ops(), 3)))
        print("HR: " + str((self.cum_home_runs_L+self.cum_home_runs_R)))
              
    def hits_L(self):
        """cumulative hits of a batter against lefties
        
        Returns:
            int: number of hits of batter agianst lefties
        """
        return(self.cum_singles_L+self.cum_doubles_L+self.cum_triples_L+self.cum_home_runs_L)
    
    def avg_L(self):
        """Calculates avg of a player against lefties
        
        Returns:
            float: batting average in decimal form of batter against lefties
        """
        if((self.cum_pa_L-self.cum_walks_L)>0):
            return(round(float((self.hits_L())/(self.cum_pa_L-self.cum_walks_L)), 3))
        else:
            return 0    
              
    def obp_L(self):
        """Calculates on base percentage of a player against lefties
        
        Returns:
            float: opb in decimal form of batter against lefties
        """
        if(self.cum_pa_L>0):
            return(round((self.hits_L()+self.cum_walks_L)/self.cum_pa_L, 3))
        else:
            return 0
              
    def slg_L(self):
        """Calculates slugging percentage of a player against lefties
        
        Returns:
            float: slg in decimal form of batter against lefties
            """
        if((self.cum_pa_L-self.cum_walks_L)>0):
            return(round(((self.cum_singles_L+2*self.cum_doubles_L+
                           3*self.cum_triples_L+4*self.cum_home_runs_L))/
                           (self.cum_pa_L-self.cum_walks_L), 3))
        else:
            return 0
              
    def ops_L(self):
        """Calculates on base plus slugging of a player against lefties
        
        Returns:
            float: ops in decimal form of batter against lefties
        """
        return(self.slg_L() + self.obp_L())
              
    def hits_R(self):
        """Cumulative hits of a batter against righties
        
        Returns:
            int: number of hits of batter against righties
        """
        return(self.cum_singles_R+self.cum_doubles_R+self.cum_triples_R+self.cum_home_runs_R)
    
    def avg_R(self):
        """Calculates avg of a player against righties
        
        Returns:
            float: batting average in decimal form of batter against righties
        """
        if((self.cum_pa_R-self.cum_walks_R)>0):
            return(round(float((self.hits_R())/(self.cum_pa_R-self.cum_walks_R)), 3))
        else:
            return 0    
              
    def obp_R(self):
        """Calculates on base percentage of a player against righties
        
        Returns:
            float: opb in decimal form of batter against righties
        """
        if(self.cum_pa_R>0):
            return(round((self.hits_R()+self.cum_walks_R)/self.cum_pa_R, 3))
        else:
            return 0
              
    def slg_R(self):
        """Calculates slugging percentage of a player against righties
        
        Returns:
            float: slg in decimal form of batter against righties
        """
        if((self.cum_pa_R-self.cum_walks_R)>0):
            return(round(((self.cum_singles_R2*self.cum_doubles_R+
                           3*self.cum_triples_R+4*self.cum_home_runs_R))/
                           (self.cum_pa_R-self.cum_walks_R), 3))
        else:
            return 0
              
    def ops_R(self):
        """Calculates on base plus slugging of a player against righties
        
        Returns:
            float: ops in decimal form of batter against righties
        """
        
        return(self.slg_R() + self.obp_R())
              
class Pitcher_new(Batter_new):
    """Class that holds information for a pitcher. Many attributes are similar to those from the Batter class.
    
    Attributes:

    """
    def __init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, hbp_L, so_L, singles_R,
                 doubles_R, triples_R, home_runs_R, walks_R, hbp_R, so_R,
                 singles_Home, doubles_Home, triples_Home, home_runs_Home, walks_Home, hbp_Home, so_Home, singles_Away, 
                 doubles_Away, triples_Away, home_runs_Away, walks_Away, hbp_Away, so_Away, sb_rate, sb_success):
        """Initializes values for this class
        
        Args:
            team: team name of pitcher
            name: pitcher name
            hand: Pitching arm of pitcher(L, R)
            singles_L: fraction of plate appearances ending in a single vs. lefties
            doubles_L: fraction of plate appearances ending in a double vs. lefties
            triples_L: fraction of plate appearances ending in a triple vs. lefties
            home_runs_L: fraction of plate appearances ending in a home run vs. lefties
            walks_L: fraction of plate appearances ending in a walk vs. lefties
            hbp_L: fraction of plate appearances ending in a hbp vs. lefties
            singles_RL: fraction of plate appearances ending in a single vs. righties
            doubles_R: fraction of plate appearances ending in a double vs. righties
            triples_R: fraction of plate appearances ending in a triple vs. righties
            home_runs_R: fraction of plate appearances ending in a home run vs. righties
            walks_R: fraction of plate appearances ending in a walk vs. righties
            hbp_R: fraction of plate appearances ending in a hbp vs. righties
            sb_rate: approximate percentage of time opposing player will steal base given opportunity (set to 0)
            sb_success: fraction of opponent stolen base attempts ending in success (set to 0)
        """
        
        Batter_new.__init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, hbp_L, so_L,
                        singles_R, doubles_R, triples_R, home_runs_R, walks_R, hbp_R, so_R,
                        singles_Home, doubles_Home, triples_Home, home_runs_Home, walks_Home, hbp_Home, so_Home, singles_Away, 
                        doubles_Away, triples_Away, home_runs_Away, walks_Away, hbp_Away, so_Away, 0, 0)
        self.cum_outs_L = 0
        self.cum_outs_R = 0
        self.cum_wp = 0
        self.current_bf = 0
        self.current_outs = 0
        self.current_runs = 0
        self.finished_half = False
        self.days_off = 100
        self.consecutive_days = 0
    
    def cum_outs(self, hand):
        """Cumulative outs of a pitcher against batters of certain handedness
        
        Args:
            hand: handedness of batter pitcher is facing
        Returns:
            float: value for cumulative outs
        """
        if(hand=='L'):
            return self.cum_outs_L
        elif(hand=='R'):
            return self.cum_outs_R
    
    def ip(self):
        """Calculates innings pitched of a pitcher
        
        Returns:
            float: Innings pitched in normal form(ie. 5.1 for 5 and a third innings pitched)
        """
        ip = (self.cum_outs_L+self.cum_outs_R+self.cum_cs)/3.0
        ip = math.modf(ip)
        ip = round((ip[0]/3.33333 + ip[1]), 2)
        return(ip)  
    
    def era(self):
        """Calculates ERA of a pitcher
        
        Returns:
            float: ERA in decimal form
        """
        if((self.cum_outs_L+self.cum_outs_R)==0):
            return 0
        else:
            return(round((27*self.cum_runs/(self.cum_outs_L+self.cum_outs_R+self.cum_cs)), 2))
    
    def whip(self):
        """Calculates WHIP of a pitcher
        
        Returns:
            float: WHIP in decimal form
        """
        if((self.cum_outs_L+self.cum_outs_R)==0):
            return 0
        else:
            return(round((((self.cum_walks_L+self.cum_walks_R)+self.hits())/
                          ((self.cum_outs_L+self.cum_outs_R+self.cum_cs)/3)), 2))

class Team_new:
    """Class that holds information for a team.
    
    Attributes:

    """
    def __init__(self, city, league, batter_df, pitcher_df, roster, batter_stats, pitcher_stats, lineup_dict=None):
        """Initializes values for this class
        
        Args:
            city: city of team (ex. 'Atlanta')
            league: league of team ('AL' or 'NL')
            batter_df: dataframe of batters on a team and their 2019 stats
            pitcher_df: dataframe of pitchers on a team and their 2019 stats
            starters: dict with names of batters in projected starting lineup, rotation, and the closer
            relievers_df: dataframe with relievers of a team and their projected innings pitched
        """
        self.city = city
        self.league = league
        self.batter_df = batter_df
        self.batter_dict= {}
        self.pitcher_df = pitcher_df
        self.pitcher_dict = {}
        self.roster = roster
        self.batter_stats = batter_stats
        self.pitcher_stats = pitcher_stats
        self.lineup_dict = lineup_dict
        self.pitchers = []   # List of pithcers on a team
        self.rotation = []   # Team's starting rotation
        self.bullpen = []    # Team's bullpen  
        self.starter = None   # Starter for a team
        self.closer = None    # Clsoer for a team
        self.batters = []    # List of batters on a team
        self.lineup = [None]*9    # A team's lineup
        self.bench = []      # Bench for a team
        self.current_pitcher_list = []     # pitchers who have already pitched in a given game
        self.available_pitchers_list = []   # Pitchers who have not pitched yet in a given game
        self.batting_index = 0   # Where team is at in lineup
        self.runners = [None, None, None, 0]    # first base-home plate. 0 for base empty, 1 for runner on base
        self.runs = 0   # Number of runs for team in a given game
        self.runners_dict = {}
        self.game_no = 0    # game number
        self.wins = 0
        self.losses = 0
        self.cum_runs = 0
        self.cum_runs_allowed = 0
        
        avg_singles_L = self.batter_stats['singles_L'].mean()/self.batter_stats['PA_L'].mean()
        avg_doubles_L = self.batter_stats['doubles_L'].mean()/self.batter_stats['PA_L'].mean()
        avg_triples_L = self.batter_stats['triples_L'].mean()/self.batter_stats['PA_L'].mean()
        avg_home_runs_L = self.batter_stats['HR_L'].mean()/self.batter_stats['PA_L'].mean()
        avg_walks_L = self.batter_stats['BB_L'].mean()/self.batter_stats['PA_L'].mean()
        avg_hbp_L = self.batter_stats['HBP_L'].mean()/self.batter_stats['PA_L'].mean()
        avg_so_L = self.batter_stats['SO_L'].mean()/self.batter_stats['PA_L'].mean()
        
        avg_singles_R = self.batter_stats['singles_R'].mean()/self.batter_stats['PA_R'].mean()
        avg_doubles_R = self.batter_stats['doubles_R'].mean()/self.batter_stats['PA_R'].mean()
        avg_triples_R = self.batter_stats['triples_R'].mean()/self.batter_stats['PA_R'].mean()
        avg_home_runs_R = self.batter_stats['HR_R'].mean()/self.batter_stats['PA_R'].mean()
        avg_walks_R = self.batter_stats['BB_R'].mean()/self.batter_stats['PA_R'].mean()
        avg_hbp_R = self.batter_stats['HBP_R'].mean()/self.batter_stats['PA_R'].mean()
        avg_so_R = self.batter_stats['SO_R'].mean()/self.batter_stats['PA_R'].mean()
        
        avg_singles_Home = self.batter_stats['singles_Home'].mean()/self.batter_stats['PA_Home'].mean()
        avg_doubles_Home = self.batter_stats['doubles_Home'].mean()/self.batter_stats['PA_Home'].mean()
        avg_triples_Home = self.batter_stats['triples_Home'].mean()/self.batter_stats['PA_Home'].mean()
        avg_home_runs_Home = self.batter_stats['HR_Home'].mean()/self.batter_stats['PA_Home'].mean()
        avg_walks_Home = self.batter_stats['BB_Home'].mean()/self.batter_stats['PA_Home'].mean()
        avg_hbp_Home = self.batter_stats['HBP_Home'].mean()/self.batter_stats['PA_Home'].mean()
        avg_so_Home = self.batter_stats['SO_Home'].mean()/self.batter_stats['PA_Home'].mean()
        
        avg_singles_Away = self.batter_stats['singles_Away'].mean()/self.batter_stats['PA_Away'].mean()
        avg_doubles_Away = self.batter_stats['doubles_Away'].mean()/self.batter_stats['PA_Away'].mean()
        avg_triples_Away = self.batter_stats['triples_Away'].mean()/self.batter_stats['PA_Away'].mean()
        avg_home_runs_Away = self.batter_stats['HR_Away'].mean()/self.batter_stats['PA_Away'].mean()
        avg_walks_Away = self.batter_stats['BB_Away'].mean()/self.batter_stats['PA_Away'].mean()
        avg_hbp_Away = self.batter_stats['HBP_Away'].mean()/self.batter_stats['PA_Away'].mean()
        avg_so_Away = self.batter_stats['SO_Away'].mean()/self.batter_stats['PA_Away'].mean()

        for index, batter in self.batter_df.iterrows():   # adding values to each batter and instantiating Batter class
            singles_L = 2*batter['singles_L']/batter['PA_L']-avg_singles_L
            doubles_L = 2*batter['doubles_L']/batter['PA_L']-avg_doubles_L
            triples_L = 2*batter['triples_L']/batter['PA_L']-avg_triples_L
            home_runs_L = 2*batter['HR_L']/batter['PA_L']-avg_home_runs_L
            walks_L = 2*batter['BB_L']/batter['PA_L']-avg_walks_L
            hbp_L = 2*batter['HBP_L']/batter['PA_L']-avg_hbp_L
            so_L = 2*batter['SO_L']/batter['PA_L']-avg_so_L
            
            singles_R = 2*batter['singles_R']/batter['PA_R']-avg_singles_R
            doubles_R = 2*batter['doubles_R']/batter['PA_R']-avg_doubles_R
            triples_R = 2*batter['triples_R']/batter['PA_R']-avg_triples_R
            home_runs_R = 2*batter['HR_R']/batter['PA_R']-avg_home_runs_R
            walks_R = 2*batter['BB_R']/batter['PA_R']-avg_walks_R
            hbp_R = 2*batter['HBP_R']/batter['PA_R']-avg_hbp_R
            so_R = 2*batter['SO_R']/batter['PA_R']-avg_so_R
            
            singles_Home = 2*batter['singles_Home']/batter['PA_Home']-avg_singles_Home
            doubles_Home = 2*batter['doubles_Home']/batter['PA_Home']-avg_doubles_Home
            triples_Home = 2*batter['triples_Home']/batter['PA_Home']-avg_triples_Home
            home_runs_Home = 2*batter['HR_Home']/batter['PA_Home']-avg_home_runs_Home
            walks_Home = 2*batter['BB_Home']/batter['PA_Home']-avg_walks_Home
            hbp_Home = 2*batter['HBP_Home']/batter['PA_Home']-avg_hbp_Home
            so_Home = 2*batter['SO_Home']/batter['PA_Home']-avg_so_Home
            
            singles_Away = 2*batter['singles_Away']/batter['PA_Away']-avg_singles_Away
            doubles_Away = 2*batter['doubles_Away']/batter['PA_Away']-avg_doubles_Away
            triples_Away = 2*batter['triples_Away']/batter['PA_Away']-avg_triples_Away
            home_runs_Away = 2*batter['HR_Away']/batter['PA_Away']-avg_home_runs_Away
            walks_Away = 2*batter['BB_Away']/batter['PA_Away']-avg_walks_Away
            hbp_Away = 2*batter['HBP_Away']/batter['PA_Away']-avg_hbp_Away
            so_Away = 2*batter['SO_Away']/batter['PA_Away']-avg_so_Away
            
            current_batter = Batter_new(self.city, batter['Name'], batter['Bats'], singles_L, doubles_L, triples_L, 
                                home_runs_L, walks_L, hbp_L, so_L, singles_R, doubles_R, triples_R, 
                                home_runs_R, walks_R, hbp_R, so_R, singles_Home, doubles_Home, triples_Home, 
                                home_runs_Home, walks_Home, hbp_Home, so_Home, singles_Away, doubles_Away, triples_Away, 
                                home_runs_Away, walks_Away, hbp_Away, so_Away, 0, 0)

            if batter['Name'] in self.lineup_dict['lineup']:   # Adding batters to lineup
                self.lineup[self.lineup_dict['lineup'].index(batter['Name'])] = current_batter
                """
                print("name", current_batter.name)
                print("1b_L", current_batter.singles_L)
                print('2b_L', current_batter.doubles_L)
                print('3b_L', current_batter.triples_L)
                print('hr_l', current_batter.home_runs_L)
                print('bb_l', current_batter.walks_L)
                print('hbp_l', current_batter.hbp_L)
                print('so_l', current_batter.so_L)
                print("1b_home", current_batter.singles_Home)
                print('2b_home', current_batter.doubles_Home)
                print('3b_home', current_batter.triples_Home)
                print('hr_home', current_batter.home_runs_Home)
                print('bb_home', current_batter.walks_Home)
                print('hbp_home', current_batter.hbp_Home)
                print('so_home', current_batter.so_Home)
                """
            else:  # Adding batters not in lineup to bench
                self.bench.append(current_batter)
            
            self.batters.append(current_batter)
            self.batter_dict.update({batter['Name']: current_batter})
            
        for num in range(len(self.lineup)-1):  # If normal batter in lineup is not in dataset, add fake batter
            if self.lineup[num]==None:
                self.lineup[num] = Batter_new(self.city, 'extra' + str(num), 'R', .13, .04, .01, .02, 
                                              .03, .25, .13, .04, .01, .02, .03, .25, .02, .01, .13, .04, .01, .02, 
                                              .03, .25, .13, .04, .01, .02, .03, .25, .02, .01, 0, 0)
            else:
                print("batter_name", self.lineup[num].name)
            
        if(self.lineup[8]==None):    # Adding poor hitting hypothetical pitcher to lineup
            self.lineup[8] = Batter_new(self.city, 'pitcher', 'R', .06, .01, -.05, -.03, -.03, 0, .5, .06, .01, 
                                                  -.05, -.03, -.03, 0, .5, .06, .01, -.05, -.03, -.03, 0, .5, .06, .01, 
                                                  -.05, -.03, -.03, 0, .5, 0, 0)
        
        avg_singles_L = self.pitcher_stats['singles_L'].mean()/self.pitcher_stats['TBF_L'].mean()
        avg_doubles_L = self.pitcher_stats['doubles_L'].mean()/self.pitcher_stats['TBF_L'].mean()
        avg_triples_L = self.pitcher_stats['triples_L'].mean()/self.pitcher_stats['TBF_L'].mean()
        avg_home_runs_L = self.pitcher_stats['HR_L'].mean()/self.pitcher_stats['TBF_L'].mean()
        avg_walks_L = self.pitcher_stats['BB_L'].mean()/self.pitcher_stats['TBF_L'].mean()
        avg_hbp_L = self.pitcher_stats['HBP_L'].mean()/self.pitcher_stats['TBF_L'].mean()
        avg_so_L = self.pitcher_stats['SO_L'].mean()/self.pitcher_stats['TBF_L'].mean()
        
        avg_singles_R = self.pitcher_stats['singles_R'].mean()/self.pitcher_stats['TBF_R'].mean()
        avg_doubles_R = self.pitcher_stats['doubles_R'].mean()/self.pitcher_stats['TBF_R'].mean()
        avg_triples_R = self.pitcher_stats['triples_R'].mean()/self.pitcher_stats['TBF_R'].mean()
        avg_home_runs_R = self.pitcher_stats['HR_R'].mean()/self.pitcher_stats['TBF_R'].mean()
        avg_walks_R = self.pitcher_stats['BB_R'].mean()/self.pitcher_stats['TBF_R'].mean()
        avg_hbp_R = self.pitcher_stats['HBP_R'].mean()/self.pitcher_stats['TBF_R'].mean()
        avg_so_R = self.pitcher_stats['SO_R'].mean()/self.pitcher_stats['TBF_R'].mean()  
        
        avg_singles_Home = self.pitcher_stats['singles_Home'].mean()/self.pitcher_stats['TBF_Home'].mean()
        avg_doubles_Home = self.pitcher_stats['doubles_Home'].mean()/self.pitcher_stats['TBF_Home'].mean()
        avg_triples_Home = self.pitcher_stats['triples_Home'].mean()/self.pitcher_stats['TBF_Home'].mean()
        avg_home_runs_Home = self.pitcher_stats['HR_Home'].mean()/self.pitcher_stats['TBF_Home'].mean()
        avg_walks_Home = self.pitcher_stats['BB_Home'].mean()/self.pitcher_stats['TBF_Home'].mean()
        avg_hbp_Home = self.pitcher_stats['HBP_Home'].mean()/self.pitcher_stats['TBF_Home'].mean()
        avg_so_Home = self.pitcher_stats['SO_Home'].mean()/self.pitcher_stats['TBF_Home'].mean()
        
        avg_singles_Away = self.pitcher_stats['singles_Away'].mean()/self.pitcher_stats['TBF_Away'].mean()
        avg_doubles_Away = self.pitcher_stats['doubles_Away'].mean()/self.pitcher_stats['TBF_Away'].mean()
        avg_triples_Away = self.pitcher_stats['triples_Away'].mean()/self.pitcher_stats['TBF_Away'].mean()
        avg_home_runs_Away = self.pitcher_stats['HR_Away'].mean()/self.pitcher_stats['TBF_Away'].mean()
        avg_walks_Away = self.pitcher_stats['BB_Away'].mean()/self.pitcher_stats['TBF_Away'].mean()
        avg_hbp_Away = self.pitcher_stats['HBP_Away'].mean()/self.pitcher_stats['TBF_Away'].mean()
        avg_so_Away = self.pitcher_stats['SO_Away'].mean()/self.pitcher_stats['TBF_Away'].mean()
        
            
        for index, pitcher in pitcher_df.iterrows(): # adding values to each pitcher and instantiating Pitcher class
            singles_L = 2*pitcher['singles_L']/pitcher['TBF_L']-avg_singles_L
            doubles_L = 2*pitcher['doubles_L']/pitcher['TBF_L']-avg_doubles_L
            triples_L = 2*pitcher['triples_L']/pitcher['TBF_L']-avg_triples_L
            home_runs_L = 2*pitcher['HR_L']/pitcher['TBF_L']-avg_home_runs_L
            walks_L = 2*pitcher['BB_L']/pitcher['TBF_L']-avg_walks_L
            hbp_L = 2*pitcher['HBP_L']/pitcher['TBF_L']-avg_hbp_L
            so_L = 2*pitcher['SO_L']/pitcher['TBF_L']-avg_so_L
            
            singles_R = 2*pitcher['singles_R']/pitcher['TBF_R']-avg_singles_R
            doubles_R = 2*pitcher['doubles_R']/pitcher['TBF_R']-avg_doubles_R
            triples_R = 2*pitcher['triples_R']/pitcher['TBF_R']-avg_triples_R
            home_runs_R = 2*pitcher['HR_R']/pitcher['TBF_R']-avg_home_runs_R
            walks_R = 2*pitcher['BB_R']/pitcher['TBF_R']-avg_walks_R
            hbp_R = 2*pitcher['HBP_R']/pitcher['TBF_R']-avg_hbp_R
            so_R = 2*pitcher['SO_R']/pitcher['TBF_R']-avg_so_R
            
            singles_Home = 2*pitcher['singles_Home']/pitcher['TBF_Home']-avg_singles_Home
            doubles_Home = 2*pitcher['doubles_Home']/pitcher['TBF_Home']-avg_doubles_Home
            triples_Home = 2*pitcher['triples_Home']/pitcher['TBF_Home']-avg_triples_Home
            home_runs_Home = 2*pitcher['HR_Home']/pitcher['TBF_Home']-avg_home_runs_Home
            walks_Home = 2*pitcher['BB_Home']/pitcher['TBF_Home']-avg_walks_Home
            hbp_Home = 2*pitcher['HBP_Home']/pitcher['TBF_Home']-avg_hbp_Home
            so_Home = 2*pitcher['SO_Home']/pitcher['TBF_Home']-avg_so_Home
            
            singles_Away = 2*pitcher['singles_Away']/pitcher['TBF_Away']-avg_singles_Away
            doubles_Away = 2*pitcher['doubles_Away']/pitcher['TBF_Away']-avg_doubles_Away
            triples_Away = 2*pitcher['triples_Away']/pitcher['TBF_Away']-avg_triples_Away
            home_runs_Away = 2*pitcher['HR_Away']/pitcher['TBF_Away']-avg_home_runs_Away
            walks_Away = 2*pitcher['BB_Away']/pitcher['TBF_Away']-avg_walks_Away
            hbp_Away = 2*pitcher['HBP_Away']/pitcher['TBF_Away']-avg_hbp_Away
            so_Away = 2*pitcher['SO_Away']/pitcher['TBF_Away']-avg_so_Away

            current_pitcher = Pitcher_new(self.city, pitcher['Name'], pitcher['Throws'], singles_L, doubles_L, triples_L, 
                                  home_runs_L, walks_L, hbp_L, so_L, singles_R, doubles_R, triples_R, 
                                  home_runs_R, walks_R, hbp_R, so_R, singles_Home, doubles_Home, triples_Home, 
                                  home_runs_Home, walks_Home, hbp_Home, so_Home, singles_Away, doubles_Away, triples_Away, 
                                  home_runs_Away, walks_Away, hbp_Away, so_Away, 0, 0)
            
            if 'starter' in self.lineup_dict.keys() and (pitcher['Name']==self.lineup_dict['starter'] or 
                                                         ('-' in self.lineup_dict['starter'] and pitcher['Name']==self.lineup_dict['starter'].replace('-', ' '))):  # Assigning starter to the team
                self.starter = current_pitcher

            if 'SP' in self.roster.keys() and pitcher['Name'] in self.roster['SP']:
                self.rotation.append(current_pitcher)
                
            if 'RP' in self.roster.keys() and pitcher['Name'] in self.roster['RP']:
                self.bullpen.append(current_pitcher)
                
            if 'Closer' in self.roster.keys() and pitcher['Name']==self.roster['Closer']:
                self.closer = current_pitcher
            
            self.pitchers.append(current_pitcher)
            self.pitcher_dict.update({pitcher['Name']: current_pitcher})
            
        if self.starter==None:
            self.starter = Pitcher_new(self.city, 'fake_starter', 'R', avg_singles_L*1.1,  avg_doubles_L*1.1, avg_triples_L*1.1, 
                                  avg_home_runs_L*1.1, avg_walks_L*1.1, avg_hbp_L*1.1, avg_so_L*0.9, avg_singles_R*1.1, avg_doubles_R*1.1, avg_triples_R*1.1, 
                                  avg_home_runs_R*1.1, avg_walks_R*1.1, avg_hbp_R*1.1, avg_so_R*0.9, avg_singles_Home*1.1, avg_doubles_Home*1.1, avg_triples_Home*1.1, 
                                  avg_home_runs_Home*1.1, avg_walks_Home*1.1, avg_hbp_Home*1.1, avg_so_Home*0.9, avg_singles_Away*1.1, avg_doubles_Away*1.1, avg_triples_Away*1.1, 
                                  avg_home_runs_Away*1.1, avg_walks_Away*1.1, avg_hbp_Away*1.1, avg_so_Away*0.9, 0, 0)
            
    def choose_pitcher(self, outs, inning, innings, runs, opp_runs, hitting_lineup, hitting_lineup_index, pitchers_lineup, pitchers_lineup_index):
        """Returns a pitcher to pitch for a team given game attributes
        
            Args:
                outs: current number of outs in inning
                runs: number of runs team has
                opp_runs: number of runs opposing team has
                hitting_lineup: batting team's lineup
                hitting_lineup_index: place in lineup of batting team
                pitchers_lineup: fielding team's opposing team's lineup
                pitchers_lineup_index: place in lineup of fielding team
            Returns:
                instance of Pitcher class to pitch in game
            """
        
        remove_pitcher = False
        self.available_pitchers_list = []
        for pitcher in self.pitchers:  # Adding pitchers who have not already pitched to a list
            if pitcher not in self.current_pitcher_list and pitcher not in self.available_pitchers_list:
                self.available_pitchers_list.append(pitcher)

        # If Pitcher hasn't faced the minimum of 3 batters or finished a hlaf inning, they remain in game
        if(self.current_pitcher.current_bf<3 and self.current_pitcher.finished_half==False): 
            return self.current_pitcher
        
        # Creating variable total that uses game factors to determine if pitcher should get taken out
        total = ((self.current_pitcher.current_bf-self.current_pitcher.current_outs)*3 + self.current_pitcher.current_outs+self.current_pitcher.current_runs*3)
        
        if(self.current_pitcher.current_runs-self.current_pitcher.current_outs/3.0>0):
            total+=(self.current_pitcher.current_runs-self.current_pitcher.current_outs/3)*2

        if(self.current_pitcher.hand==hitting_lineup[hitting_lineup_index].hand):   # If batter and pitcher have same handedness
            total-=3
            
        if(self.current_pitcher.finished_half):  # If pitcher has already finished half inning
            total+=2
        
        if(outs==0 and (self.current_pitcher in self.rotation or self.current_pitcher==self.starter)):    # If no outs yet in inning
            total+=5
            
        if(pitchers_lineup_index>4 and pitchers_lineup[8].name=='pitcher'):  # If pitcher is in lineup, spot is almost up
            total+=6
            
        if innings==7:   # Add more for pitchers in 7 inning games
            total+=6
        
        if(total<42):
            num = random.uniform(0, (60-total)*100)
        else:
            num = random.uniform(0, (90-total)*4)
        
        if(num<total):
            remove_pitcher = True
        
        # Leave current potcher in probabilistically
        if(not remove_pitcher and (self.current_pitcher in self.rotation or self.current_pitcher==self.starter) or total<=8 and self.current_pitcher not 
           in self.rotation):
            return self.current_pitcher
        else:  # If current pitcher has total value greater than threshold, find new reliever
            
            if(inning==innings-1 and outs>0 and self.closer!=None and self.closer not in self.current_pitcher_list 
               and runs-opp_runs>0 and runs-opp_runs<3): # Put closer in game if game is close and in late innings
                self.current_pitcher = self.closer
                self.current_pitcher.cum_G+=1
                self.current_pitcher_list.append(self.current_pitcher)
                return self.current_pitcher
            elif(inning>innings-1 and self.closer!=None and self.closer not in self.current_pitcher_list and runs-opp_runs>0 and runs-opp_runs<4):
                self.current_pitcher = self.closer
                self.current_pitcher.cum_G+=1
                self.current_pitcher_list.append(self.current_pitcher)
                return self.current_pitcher
            else:
                # If game not close or game not in late innings, put non-closer reliever in game
                
                left_count = 0    # Number of lefties out of next three batters
                right_count = 0   # Number of righties out of next three batters

                for num in range(3):
                    if(hitting_lineup[(hitting_lineup_index+num)%9].hand=='L'):
                        left_count+=(1+outs-num)
                    elif(hitting_lineup[(hitting_lineup_index+num)%9].hand=='R'):
                        right_count+=(1+outs-num)

                if(left_count>right_count):  # If more of the next 3 batters are lefties, hand_indicator = 0
                    hand_indicator = 'L'
                else:                        # If more or equal of the next 3 batters are righties, hand_indicator = 1
                    hand_indicator = 'R'
                
        
                reliever_chosen = False
                wrong_hand = 0
                attempts = 0
                
                relievers_left = []
                for pitcher in self.bullpen:
                    if pitcher in self.available_pitchers_list and self.closer!=pitcher:
                        relievers_left.append(pitcher)
                      
                if len(relievers_left)==0:
                    if self.closer in self.available_pitchers_list:
                        self.current_pitcher = self.closer
                        self.current_pitcher.cum_G+=1
                        self.current_pitcher_list.append(self.current_pitcher)
                        return self.current_pitcher
                    else:

                        pitcher_chosen = False
                        attempts = 0
                        while(not pitcher_chosen and len(self.available_pitchers_list)>0 and attempts<100):
                            num = random.randint(0, len(self.available_pitchers_list)-1)
                            pitcher = self.available_pitchers_list[num]
                            if(pitcher in self.rotation and attempts<30 and pitcher.days_off<3):
                                attempts+=1
                            elif(pitcher in self.rotation and attempts<60 and pitcher.days_off<2):
                                attempts+=1
                            elif(pitcher in self.rotation and attempts<100 and pitcher.days_off<1):
                                attempts+=1
                            else:
                                self.current_pitcher = pitcher
                                self.current_pitcher.cum_G+=1
                                self.current_pitcher_list.append(self.current_pitcher)
                                return self.current_pitcher
   
                elif len(relievers_left)==1:
                    self.current_pitcher = relievers_left[0]
                    self.current_pitcher.cum_G+=1
                    self.current_pitcher_list.append(self.current_pitcher)
                    return self.current_pitcher
            
                else:
                
                    while(attempts<200):    # Trying to find a reliever

                        num = random.randint(0, len(relievers_left)-1)
                        pitcher = relievers_left[num]
                        total = 0
                        if(pitcher.hand==hand_indicator or wrong_hand>1):
                            self.current_pitcher = pitcher
                            self.current_pitcher.cum_G+=1
                            self.current_pitcher_list.append(self.current_pitcher)
                            return self.current_pitcher
                        if(wrong_hand<2):    # If pitcher chosen doesn't have same hand as most upcoming batters, will try again
                            wrong_hand+=1
                            attempts+=1
                            break


        return self.current_pitcher
           
    def restart(self, season=False):
        """Resets values for position in lineup, runners on base, and runs if a game is to be started
        """
        for pitcher in self.current_pitcher_list:
            pitcher.consecutive_days+=1
            pitcher.days_off = 0
            pitcher.current_bf = 0
            pitcher.current_outs = 0
            pitcher.current_runs = 0
            pitcher.finished_half = False
        
        for pitcher in self.available_pitchers_list:
            pitcher.days_off+=1
            pitcher.consecutive_days = 0

        self.runners_dict = {}    
        self.current_pitcher_list = []   
        self.batting_index = 0
        self.current_pitcher = None
        self.starter_pitching = True
        self.runners = [None, None, None, 0]
        self.runs = 0
        
        if season:
            self.wins = 0
            self.losses = 0
            self.cum_runs = 0
            self.cum_runs_allowed = 0
            
            for batter in self.batters:
                batter.cum_singles_L = 0
                batter.cum_doubles_L = 0
                batter.cum_triples_L = 0
                batter.cum_home_runs_L = 0
                batter.cum_walks_L = 0
                batter.cum_hbp_L = 0
                batter.cum_so_L = 0
                batter.cum_pa_L = 0
                batter.cum_singles_R = 0
                batter.cum_doubles_R = 0
                batter.cum_triples_R = 0
                batter.cum_home_runs_R = 0
                batter.cum_walks_R = 0
                batter.cum_hbp_R = 0
                batter.cum_so_R = 0
                batter.cum_pa_R = 0
                batter.cum_runs = 0
                batter.cum_rbi = 0
                batter.cum_sb = 0
                batter.cum_cs = 0
                batter.cum_sf = 0
                batter.cum_sh = 0
                batter.cum_gdp = 0
                batter.cum_G = 0
                batter.cum_GS = 0
                
            for pitcher in self.pitchers:
                pitcher.cum_singles_L = 0
                pitcher.cum_doubles_L = 0
                pitcher.cum_triples_L = 0
                pitcher.cum_home_runs_L = 0
                pitcher.cum_walks_L = 0
                pitcher.cum_hbp_L = 0
                pitcher.cum_so_L = 0
                pitcher.cum_pa_L = 0
                pitcher.cum_singles_R = 0
                pitcher.cum_doubles_R = 0
                pitcher.cum_triples_R = 0
                pitcher.cum_home_runs_R = 0
                pitcher.cum_walks_R = 0
                pitcher.cum_hbp_R = 0
                pitcher.cum_so_R = 0
                pitcher.cum_pa_R = 0
                pitcher.cum_runs = 0
                pitcher.cum_rbi = 0
                pitcher.cum_sb = 0
                pitcher.cum_cs = 0
                pitcher.cum_sf = 0
                pitcher.cum_sh = 0
                pitcher.cum_gdp = 0
                pitcher.cum_G = 0
                pitcher.cum_GS = 0
                pitcher.cum_outs_L = 0
                pitcher.cum_outs_R = 0
                pitcher.cum_wp = 0

    def reset_runners(self):
        """Resets baserunners after each half inning
        """
        self.runners = [None, None, None, 0]
    
    def cumul_stats_df(self):
        """Class that returns dataframes of stats for batters and pitchers
    
        Returns:
            batter_df: dataframe of batter stats
            pitcher_df: dataframe of pitcher stats
        """
        batter_df = pd.DataFrame(columns=['Name', 'G', 'PA', 'AB', 'H', 'singles', 'doubles', 'triples', 'HR', 'BB', 'HBP', 'SF', 
                                          'SH', 'K', 'AVG', 'OBP', 'SLG', 'OPS', 'R', 'RBI', 'SB', 'CS'])
        
        for b in self.batters:
            batter_values = [b.name, b.cum_G, b.cum_pa_L+b.cum_pa_R, b.cum_pa_L+b.cum_pa_R-
                             (b.cum_walks_L+b.cum_walks_R+b.cum_hbp_L+b.cum_hbp_R+b.cum_sf), 
                             b.hits(), b.cum_singles_L+b.cum_singles_R, b.cum_doubles_L+b.cum_doubles_R, 
                             b.cum_triples_L+b.cum_triples_R, b.cum_home_runs_L+b.cum_home_runs_R, 
                             b.cum_walks_L+b.cum_walks_R, b.cum_hbp_L+b.cum_hbp_R, b.cum_sf, b.cum_sh, b.strikeouts(), b.avg(), b.obp(), 
                             b.slg(), b.ops(), b.cum_runs, b.cum_rbi, b.cum_sb, b.cum_cs]
            batter_df = batter_df.append(pd.Series(batter_values, index=batter_df.columns), ignore_index=True)
        batter_df.sort_values(by=['PA'], ascending=False, inplace=True)
        
        pitcher_df = pd.DataFrame(columns=['Name', 'G', 'GS', 'IP', 'K', 'H', 'BB', 'HBP', 'ERA', 'WHIP', 'HR',
                                           'AVG', 'OBP', 'SLG', 'OPS', 'R', 'SB', 'CS', 'WP'])
        
        for p in self.pitchers:
            pitcher_values = [p.name, p.cum_G, p.cum_GS, p.ip(), p.strikeouts(), p.hits(), p.cum_walks_L+p.cum_walks_R, 
                              p.cum_hbp_L+p.cum_hbp_R, p.era(), p.whip(), p.cum_home_runs_L+p.cum_home_runs_R, p.avg(), 
                              p.obp(), p.slg(), p.ops(), p.cum_runs, p.cum_sb, p.cum_cs, p.cum_wp]
            pitcher_df = pitcher_df.append(pd.Series(pitcher_values, index=pitcher_df.columns), ignore_index=True)
        pitcher_df.sort_values(by=['IP'], ascending=False, inplace=True) 
        
        return batter_df, pitcher_df
    
class Game_new:
    """Class that holds information for a Game.
    
    Attributes:

    """
    
    def __init__(self, away_team, home_team, innings):
        """Initializes values for this class
        
        Args:
            away_team: away team name
            home_team: home team name
        """
        self.away_team = away_team
        self.home_team = home_team
        self.innings = innings
                                
    def get_sb_wp_result(self, runners, pitcher, team, outs, num):
        """stolen base result of a batter
        
        Args:
            runners: list of runners on base
            pitcher: pitcher currently pitching
            team: batting team
            outs: number of outs
            num: random number used to determine whether or not batter should steal base
        Returns:
            result of stolen base attempt (or None if no attempt)
        """
        if(runners[0]!=None and runners[1]==None):   # If first base occupied and second base not, steal possible
            if(num>runners[0].sb_rate/1.2):    # No stolen base attempt if random number greater than runner sb_rate
                return None
            else:    # Stolen base attempt
                x = random.uniform(0,1)
                if(x<runners[0].sb_success):  # Successful stolen base attempt
                    runners[0].cum_sb+=1
                    pitcher.cum_sb+=1
                    runners[1] = runners[0]
                    runners[0] = None
                    return None
                        
                else:    # Batter caught stealing
                    runners[0].cum_cs+=1
                    pitcher.cum_cs+=1
                    runners[0] = None
                    return "out"
        elif(runners[0]!=None and runners[1]!=None):   # If first and second base occupied
            if(num>runners[0].sb_rate/5):    # No stolen base attempt if random number greater than runner sb_rate
                return None
            else:    # Stolen base attempt
                x = random.uniform(0,1)
                if(x<runners[1].sb_success):  # Successful stolen base attempt
                    runners[1].cum_sb+=1
                    runners[0].cum_sb+=1
                    pitcher.cum_sb+=2
                    runners[2] = runners[1]
                    runners[1] = runners[0]
                    runners[0] = None
                    return None
                        
                else:    # Batter caught stealing
                    runners[1].cum_cs+=1
                    pitcher.cum_cs+=1
                    if(outs<2):
                        runners[0].cum_sb+=1
                        pitcher.cum_sb+=1
                        runners[1] = runners[0]
                        
                    runners[0] = None
                    return "out"
        elif(runners[0]!=None and runners[1]!=None):   # If second base occupied
            if(num>runners[0].sb_rate/5):    # No stolen base attempt if random number greater than runner sb_rate
                return None
            else:    # Stolen base attempt
                x = random.uniform(0,1)
                if(x<runners[1].sb_success):  # Successful stolen base attempt
                    runners[1].cum_sb+=1
                    pitcher.cum_sb+=1
                    runners[2] = runners[1]
                    runners[1] = None
                    return None
                        
                else:    # Batter caught stealing
                    runners[1].cum_cs+=1
                    pitcher.cum_cs+=1
                    runners[1] = None
                    return "out"
        
        wp_num = random.uniform(0,1)
        if(wp_num<.01):    # Seeing whether at bat featured wild pitch
            pitcher.cum_wp+=1
            if(runners[2]!=None):
                team.runs+=1
                runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                
            team.runners[2] = team.runners[1]
            team.runners[1] = team.runners[0]
            team.runners[0] = None
        if(wp_num<.0005):    # Seeing whether at bat featured second wild pitch   
            pitcher.cum_wp+=1
            if(runners[2]!=None):
                team.runs+=1
                runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                
            team.runners[2] = team.runners[1]
            team.runners[1] = team.runners[0]
            team.runners[0] = None          
        
    def get_result(self, batter, pitcher, outs, runners, batting_side, num):
        """Using player attributes and a random number, a result is created for a plate appearance
        
        Args:
            batter: instance of batter class
            pitcher: instance of pitcher class
            outs: number of outs in inning
            runners: List of where runners are
            num: random number between 0 and 2
        Returns:
            str: refers to result based on random number (ie "walk")
        """      
        hand_multiplier = 3/4
        field_multiplier = 1-hand_multiplier
        
        if(pitcher.hand=='L'):
            batter_against = 'L'
        elif(pitcher.hand=='R'):
            batter_against = 'R'
              
        if(batter.hand=='L'):
            pitcher_against = 'L'
        elif(batter.hand=='R'):
            pitcher_against = 'R'
        elif(batter.hand=='S'):
            if(pitcher.hand=='L'):
                pitcher_against='R'
            elif(pitcher.hand=='R'):
                pitcher_against='L'
        
        pitcher.current_bf+=1
        
        sacrifice_num = random.uniform(0,1)
        if(outs<2 and batter.name=='pitcher' and runners[2]==None and (runners[1]!=None or runners[2]==None)):    # Seeing whether a pitcher will try to lay down a sacrifice bunt
            if(sacrifice_num<.3):
                
                batter.cum_sh+=1
                
                if(batter_against=='L'):
                    batter.cum_pa_L+=1
                else:
                    batter.cum_pa_R+=1
                    
                if(pitcher_against=='L'):
                    pitcher.cum_pa_L+=1
                    pitcher.cum_outs_L+=1
                else:
                    pitcher.cum_pa_R+=1
                    pitcher.cum_outs_R+=1
                
                return 'sh'
            elif(sacrifice_num<.45):
                
                if(batter_against=='L'):
                    batter.cum_pa_L+=1
                else:
                    batter.cum_pa_R+=1
                    
                if(pitcher_against=='L'):
                    pitcher.cum_pa_L+=1
                    pitcher.cum_outs_L+=1
                else:
                    pitcher.cum_pa_R+=1
                    pitcher.cum_outs_R+=1
                
                return 'out'
            
        if(batter_against=='L' and pitcher_against=='L' and batting_side=='Home'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_L+pitcher.singles_L)*hand_multiplier + (batter.singles_Home+pitcher.singles_Away)*field_multiplier):
                batter.cum_singles_L+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away)*field_multiplier):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away)*field_multiplier):
                batter.cum_triples_L+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L
                      +batter.triples_L+pitcher.triples_L+batter.home_runs_L+pitcher.home_runs_L)*hand_multiplier
                    + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                    +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away)*field_multiplier):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L
                     +batter.home_runs_L+pitcher.home_runs_L+batter.walks_L+pitcher.walks_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                     +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away)*field_multiplier):
                batter.cum_walks_L+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L
                     +batter.home_runs_L+pitcher.home_runs_L+batter.walks_L+pitcher.walks_L+batter.hbp_L+pitcher.hbp_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home
                     +pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home
                     +pitcher.walks_Away+batter.hbp_Home+pitcher.hbp_Away)*field_multiplier):
                batter.cum_hbp_L+=1
                pitcher.cum_hbp_L+=1
                return "hbp"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L
                     +batter.home_runs_L+pitcher.home_runs_L+batter.walks_L+pitcher.walks_L+batter.hbp_L+pitcher.hbp_L+batter.so_L+pitcher.so_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away
                     +batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away
                     +batter.hbp_Home+pitcher.hbp_Away+batter.so_Home+pitcher.so_Away)*field_multiplier):
                batter.cum_so_L+=1                
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            else:
                 
                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
                
        elif(batter_against=='L' and pitcher_against=='L' and batting_side=='Away'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_L+pitcher.singles_L)*hand_multiplier + (batter.singles_Away+pitcher.singles_Home)*field_multiplier):
                batter.cum_singles_L+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home)*field_multiplier):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home)*field_multiplier):
                batter.cum_triples_L+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L
                      +batter.triples_L+pitcher.triples_L+batter.home_runs_L+pitcher.home_runs_L)*hand_multiplier
                    + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                    +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home)*field_multiplier):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L
                     +batter.home_runs_L+pitcher.home_runs_L+batter.walks_L+pitcher.walks_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                     +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home)*field_multiplier):
                batter.cum_walks_L+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L
                     +batter.home_runs_L+pitcher.home_runs_L+batter.walks_L+pitcher.walks_L+batter.hbp_L+pitcher.hbp_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away
                     +pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away
                     +pitcher.walks_Home+batter.hbp_Away+pitcher.hbp_Home)*field_multiplier):
                batter.cum_hbp_L+=1
                pitcher.cum_hbp_L+=1
                return "hbp"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L
                     +batter.home_runs_L+pitcher.home_runs_L+batter.walks_L+pitcher.walks_L+batter.hbp_L+pitcher.hbp_L+batter.so_L+pitcher.so_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home
                     +batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home
                     +batter.hbp_Away+pitcher.hbp_Home+batter.so_Away+pitcher.so_Home)*field_multiplier):
                batter.cum_so_L+=1                
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            else:
                 
                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
                
        elif(batter_against=='L' and pitcher_against=='R' and batting_side=='Home'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_L+pitcher.singles_R)*hand_multiplier + (batter.singles_Home+pitcher.singles_Away)*field_multiplier):
                batter.cum_singles_L+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away)*field_multiplier):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away)*field_multiplier):
                batter.cum_triples_L+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R
                      +batter.triples_L+pitcher.triples_R+batter.home_runs_L+pitcher.home_runs_R)*hand_multiplier
                    + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                    +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away)*field_multiplier):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R
                     +batter.home_runs_L+pitcher.home_runs_R+batter.walks_L+pitcher.walks_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                     +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away)*field_multiplier):
                batter.cum_walks_L+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R
                     +batter.home_runs_L+pitcher.home_runs_R+batter.walks_L+pitcher.walks_R+batter.hbp_L+pitcher.hbp_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home
                     +pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home
                     +pitcher.walks_Away+batter.hbp_Home+pitcher.hbp_Away)*field_multiplier):
                batter.cum_hbp_L+=1
                pitcher.cum_hbp_R+=1
                return "hbp"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R
                     +batter.home_runs_L+pitcher.home_runs_R+batter.walks_L+pitcher.walks_R+batter.hbp_L+pitcher.hbp_R+batter.so_L+pitcher.so_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away
                     +batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away
                     +batter.hbp_Home+pitcher.hbp_Away+batter.so_Home+pitcher.so_Away)*field_multiplier):
                batter.cum_so_L+=1                
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            else:
                 
                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_R+=1
                    return "out"
                
        elif(batter_against=='L' and pitcher_against=='R' and batting_side=='Away'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_L+pitcher.singles_R)*hand_multiplier + (batter.singles_Away+pitcher.singles_Home)*field_multiplier):
                batter.cum_singles_L+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home)*field_multiplier):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home)*field_multiplier):
                batter.cum_triples_L+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R
                      +batter.triples_L+pitcher.triples_R+batter.home_runs_L+pitcher.home_runs_R)*hand_multiplier
                    + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                    +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home)*field_multiplier):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R
                     +batter.home_runs_L+pitcher.home_runs_R+batter.walks_L+pitcher.walks_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                     +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home)*field_multiplier):
                batter.cum_walks_L+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R
                     +batter.home_runs_L+pitcher.home_runs_R+batter.walks_L+pitcher.walks_R+batter.hbp_L+pitcher.hbp_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away
                     +pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away
                     +pitcher.walks_Home+batter.hbp_Away+pitcher.hbp_Home)*field_multiplier):
                batter.cum_hbp_L+=1
                pitcher.cum_hbp_R+=1
                return "hbp"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R
                     +batter.home_runs_L+pitcher.home_runs_R+batter.walks_L+pitcher.walks_R+batter.hbp_L+pitcher.hbp_R+batter.so_L+pitcher.so_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home
                     +batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home
                     +batter.hbp_Away+pitcher.hbp_Home+batter.so_Away+pitcher.so_Home)*field_multiplier):
                batter.cum_so_L+=1                
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            else:
                 
                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_R+=1
                    return "out"
                
        elif(batter_against=='R' and pitcher_against=='L' and batting_side=='Home'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_R+pitcher.singles_L)*hand_multiplier + (batter.singles_Home+pitcher.singles_Away)*field_multiplier):
                batter.cum_singles_R+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away)*field_multiplier):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away)*field_multiplier):
                batter.cum_triples_R+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L
                      +batter.triples_R+pitcher.triples_L+batter.home_runs_R+pitcher.home_runs_L)*hand_multiplier
                    + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                    +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away)*field_multiplier):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L
                     +batter.home_runs_R+pitcher.home_runs_L+batter.walks_R+pitcher.walks_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                     +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away)*field_multiplier):
                batter.cum_walks_R+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L
                     +batter.home_runs_R+pitcher.home_runs_L+batter.walks_R+pitcher.walks_L+batter.hbp_R+pitcher.hbp_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home
                     +pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home
                     +pitcher.walks_Away+batter.hbp_Home+pitcher.hbp_Away)*field_multiplier):
                batter.cum_hbp_R+=1
                pitcher.cum_hbp_L+=1
                return "hbp"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L
                     +batter.home_runs_R+pitcher.home_runs_L+batter.walks_R+pitcher.walks_L+batter.hbp_R+pitcher.hbp_L+batter.so_R+pitcher.so_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away
                     +batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away
                     +batter.hbp_Home+pitcher.hbp_Away+batter.so_Home+pitcher.so_Away)*field_multiplier):
                batter.cum_so_R+=1                
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            else:
                 
                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
                
        elif(batter_against=='R' and pitcher_against=='L' and batting_side=='Away'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_R+pitcher.singles_L)*hand_multiplier + (batter.singles_Away+pitcher.singles_Home)*field_multiplier):
                batter.cum_singles_R+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home)*field_multiplier):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home)*field_multiplier):
                batter.cum_triples_R+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L
                      +batter.triples_R+pitcher.triples_L+batter.home_runs_R+pitcher.home_runs_L)*hand_multiplier
                    + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                    +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home)*field_multiplier):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L
                     +batter.home_runs_R+pitcher.home_runs_L+batter.walks_R+pitcher.walks_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                     +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home)*field_multiplier):
                batter.cum_walks_R+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L
                     +batter.home_runs_R+pitcher.home_runs_L+batter.walks_R+pitcher.walks_L+batter.hbp_R+pitcher.hbp_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away
                     +pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away
                     +pitcher.walks_Home+batter.hbp_Away+pitcher.hbp_Home)*field_multiplier):
                batter.cum_hbp_R+=1
                pitcher.cum_hbp_L+=1
                return "hbp"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L
                     +batter.home_runs_R+pitcher.home_runs_L+batter.walks_R+pitcher.walks_L+batter.hbp_R+pitcher.hbp_L+batter.so_R+pitcher.so_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home
                     +batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home
                     +batter.hbp_Away+pitcher.hbp_Home+batter.so_Away+pitcher.so_Home)*field_multiplier):
                batter.cum_so_R+=1                
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            else:

                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
                
        elif(batter_against=='R' and pitcher_against=='R' and batting_side=='Home'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_R+pitcher.singles_R)*hand_multiplier + (batter.singles_Home+pitcher.singles_Away)*field_multiplier):
                batter.cum_singles_R+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away)*field_multiplier):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away)*field_multiplier):
                batter.cum_triples_R+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R
                      +batter.triples_R+pitcher.triples_R+batter.home_runs_R+pitcher.home_runs_R)*hand_multiplier
                    + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                    +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away)*field_multiplier):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R
                     +batter.home_runs_R+pitcher.home_runs_R+batter.walks_R+pitcher.walks_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                     +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away)*field_multiplier):
                batter.cum_walks_R+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R
                     +batter.home_runs_R+pitcher.home_runs_R+batter.walks_R+pitcher.walks_R+batter.hbp_R+pitcher.hbp_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home
                     +pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home
                     +pitcher.walks_Away+batter.hbp_Home+pitcher.hbp_Away)*field_multiplier):
                batter.cum_hbp_R+=1
                pitcher.cum_hbp_R+=1
                return "hbp"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R
                     +batter.home_runs_R+pitcher.home_runs_R+batter.walks_R+pitcher.walks_R+batter.hbp_R+pitcher.hbp_R+batter.so_R+pitcher.so_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away
                     +batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away
                     +batter.hbp_Home+pitcher.hbp_Away+batter.so_Home+pitcher.so_Away)*field_multiplier):
                batter.cum_so_R+=1                
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            else:
              
                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_R+=1
                    return "out"
                
        elif(batter_against=='R' and pitcher_against=='R' and batting_side=='Away'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_R+pitcher.singles_R)*hand_multiplier + (batter.singles_Away+pitcher.singles_Home)*field_multiplier):
                batter.cum_singles_R+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home)*field_multiplier):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home)*field_multiplier):
                batter.cum_triples_R+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R
                      +batter.triples_R+pitcher.triples_R+batter.home_runs_R+pitcher.home_runs_R)*hand_multiplier
                    + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                    +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home)*field_multiplier):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R
                     +batter.home_runs_R+pitcher.home_runs_R+batter.walks_R+pitcher.walks_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                     +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home)*field_multiplier):
                batter.cum_walks_R+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R
                     +batter.home_runs_R+pitcher.home_runs_R+batter.walks_R+pitcher.walks_R+batter.hbp_R+pitcher.hbp_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away
                     +pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away
                     +pitcher.walks_Home+batter.hbp_Away+pitcher.hbp_Home)*field_multiplier):
                batter.cum_hbp_R+=1
                pitcher.cum_hbp_R+=1
                return "hbp"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R
                     +batter.home_runs_R+pitcher.home_runs_R+batter.walks_R+pitcher.walks_R+batter.hbp_R+pitcher.hbp_R+batter.so_R+pitcher.so_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home
                     +batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home
                     +batter.hbp_Away+pitcher.hbp_Home+batter.so_Away+pitcher.so_Home)*field_multiplier):
                batter.cum_so_R+=1                
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            else:
                 
                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_R+=1
                    return "out"
                            
    def move_runners(self, team, batter, pitcher, outs, result):
        """Based on the result of a batter's plate appearance, runners are moved and runs may be added to team's total 
        
        Args:
            team: instance team that the batter was on
            batter who is hitting
            pitcher: pitcher pitching to team
            outs: number of outs
            result: str of result for batter (ie "walk")
        """
        made_out = False

        if(pitcher.hand=='L'):
            batter_against = 'L'
        elif(pitcher.hand=='R'):
            batter_against = 'R'
              
        if(batter.hand=='L'):
            pitcher_against = 'L'
        elif(batter.hand=='R'):
            pitcher_against = 'R'
        elif(batter.hand=='S'):
            if(pitcher.hand=='L'):
                pitcher_against='R'
            elif(pitcher.hand=='R'):
                pitcher_against='L'
        
        if(result=='single'):            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                team.runners[2] = None
                
            if(team.runners[1]!=None): 
                num = random.uniform(0, 100)    # Random number that will be used to move runners probabilistically
                
                #if(num<((batter.bsr*2)+45+(outs*10))):
                if(num<50):
                    new_num = random.uniform(0,100)
                    if(new_num<3):
                        if(pitcher_against=='L'):
                            made_out = True
                        else:
                            pitcher.cum_outs_R+=1
                            made_out = True
                    else:
                        team.runners[1].cum_runs+=1
                        team.runners_dict[team.runners[1]].cum_runs+=1
                        team.runners_dict[team.runners[1]].current_runs+=1
                        batter.cum_rbi+=1
                        team.runners[3]+=1
                else:
                    team.runners[2] = team.runners[1]
                    
                team.runners[1] = None
                
            if(team.runners[0]!=None and team.runners[2]==None):
                num = random.uniform(0, 100)    # Random number that will be used to move runners probabilistically

                #if(num<((batter.bsr*3)+30)):
                if(num<30):
                    new_num = random.uniform(0,100)
                    if(new_num<2):
                        if(pitcher_against=='L'):
                            pitcher.cum_outs_L+=1
                            made_out = True
                        else:
                            pitcher.cum_outs_R+=1
                            made_out = True
                    else:
                        team.runners[2] = team.runners[0] 
                else:
                    team.runners[1] = team.runners[0]
            else:
                team.runners[1] = team.runners[0]
                
            team.runners[0] = batter
            team.runners_dict.update({batter: pitcher})

        elif(result=='double'):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.runners_dict[team.runners[1]].cum_runs+=1
                team.runners_dict[team.runners[1]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            
            if(team.runners[0]!=None):
                num = random.uniform(0, 100)    # Random number that will be used to move runners probabilistically
                #if(num<((batter.bsr*4)+45)):
                if(num<50):
                    new_num = random.uniform(0,100)
                    if(new_num<2):
                        if(pitcher_against=='L'):
                            pitcher.cum_outs_L+=1
                            made_out = True
                        else:
                            pitcher.cum_outs_R+=1
                            made_out = True
                    else:
                        team.runners[0].cum_runs+=1
                        team.runners_dict[team.runners[0]].cum_runs+=1
                        team.runners_dict[team.runners[0]].current_runs+=1
                        batter.cum_rbi+=1
                        team.runners[3]+=1 
                else:
                    team.runners[2] = team.runners[0]
                
            team.runners[1] = batter
            team.runners[0] = None
            team.runners_dict.update({batter: pitcher})

        elif(result=='triple'):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.runners_dict[team.runners[1]].cum_runs+=1
                team.runners_dict[team.runners[1]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                team.runners_dict[team.runners[0]].cum_runs+=1
                team.runners_dict[team.runners[0]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = batter
            team.runners[1] = None
            team.runners[0] = None
            team.runners_dict.update({batter: pitcher})

        elif(result=='home_run'):
            batter.cum_runs+=1
            pitcher.cum_runs+=1
            pitcher.current_runs+=1
            batter.cum_rbi+=1
            team.runners[3]+=1
            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.runners_dict[team.runners[1]].cum_runs+=1
                team.runners_dict[team.runners[1]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                team.runners_dict[team.runners[0]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = None
            team.runners[1] = None
            team.runners[0] = None

        elif(result=='walk' or result=='hbp' or result=='sh'):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            team.runners_dict.update({batter: pitcher})
            
            if(temp_0!=None and temp_1!=None and temp_2!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            
            team.runners[0] = batter
            
            if(temp_0!=None):
                team.runners[1] = temp_0
                
                if(temp_1!=None):
                    team.runners[2] = temp_1
                    
            if(result=='sh'):   # If sacrifice is hit, batter doesn't reach base
                team.runners[0] = None
                                          
        elif(result=='gdp'):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            rand_num = random.uniform(0,1)

            if(temp_1==None and temp_2==None):    # If only runner on first
                team.runners[0] = None
            elif(temp_1!=None and temp_2==None):    # If runners on first and second
                if(rand_num<.02):   # Outs made at first and third
                    team.runners[1] = temp_0
                    team.runners[0] = None
                elif(rand_num<.08):   # Outs made at third and second
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                else:
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = None
            elif(temp_1==None and temp_2!=None):    # If runners on first and third
                if(rand_num<.01):   # Batter goes to first
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                    team.runners[2] = None
                elif(rand_num<.02):   # Batter goes to second
                    team.runners[0] = None
                    team.runners[1] = batter
                    team.runners_dict.update({batter: pitcher})
                    team.runners[2] = None
                else: # Runner scores, bases empty
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    team.runners[3]+=1
                    team.runners[2] = None
                    team.runners[1] = None
                    team.runners[0] = None
            elif(temp_1!=None and temp_2!=None):    # If bases loaded
                if(rand_num<.6):   # Outs made at second and first
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    team.runners[3]+=1
                    team.runners[0] = None
                    team.runners[1] = None
                    team.runners[2] = temp_1
                elif(rand_num<.9):   # Outs made at home and first
                    team.runners[1] = temp_0
                    team.runners[2] = temp_1
                    team.runners[0] = None
                else:    # Outs made at third and home
                    team.runners[2] = None
                    team.runners[1] = temp_0
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                                        
        elif(result=='out'):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            
            if(temp_0!=None and temp_1==None and temp_2==None):
                rand_num = random.uniform(0,1)
                
                #if(rand_num<(.3+batter.bsr/15)):   # If batter hits ground ball and reaches on fielder's choice
                if(rand_num<0.3):    
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.4):    # If batter hits ground ball and is out but runner on first moves to second
                    team.runners[1] = temp_1
                    team.runners[0] = None
            elif(temp_0==None and temp_1!=None and temp_2==None):
                rand_num = random.uniform(0,1)
                #if(rand_num<(.3+temp_1.bsr/15)):   # If batter hits sacrifice fly moving runner to third or grounder moving runner
                if(rand_num<0.3): 
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners_dict.update({batter: pitcher})
            elif(temp_0==None and temp_1==None and temp_2!=None):
                rand_num = random.uniform(0,1)
                #if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                if(rand_num<0.3): 
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    batter.cum_sf+=1
                    team.runners[3]+=1
                elif(rand_num<.4):   # If batter hits ground ball scoring runner
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    team.runners[3]+=1
            elif(temp_0!=None and temp_1!=None and temp_2==None):
                rand_num = random.uniform(0,1)
                #if(rand_num<(.1+temp_1.bsr/15)):   # If batter hits sacrifice fly moving runner to third
                if(rand_num<0.1): 
                    team.runners[2] = temp_1
                    team.runners[1] = None
                elif(rand_num<.3):    # If batter hits ground ball, guy on second goes to third, guy on first is out, batter goes to first
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.35):    # If batter hits ball, guy on second goes to third, guy on first goes to second, batter makes out
                    team.runners[2] = temp_1
                    team.runners[1] = temp_0
                    team.runners[0] = None
            elif(temp_0!=None and temp_1==None and temp_2!=None):
                rand_num = random.uniform(0,1)
                #if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                if(rand_num<0.3): 
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    batter.cum_sf+=1
                    team.runners[3]+=1
                    
                    if(rand_num<.05):
                        team.runners[1] = temp_0
                    
                    team.runners[2] = None
                elif(rand_num<.3):    # If batter hits ground ball, out made at home
                    team.runners[2] = None
                    team.runners[1] = temp_0
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                #elif(rand_num<.35+batter.bsr/15):    # If batter hits ball, guy on third scores, guy on first out, batter ends up on first
                elif(rand_num<0.35): 
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    team.runners[3]+=1
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
            elif(temp_0==None and temp_1!=None and temp_2!=None):
                rand_num = random.uniform(0,1)
                #if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                if(rand_num<0.3): 
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    batter.cum_sf+=1
                    team.runners[3]+=1
                    
                    if(rand_num<.15):
                        team.runners[2] = temp_1
                    
                    team.runners[1] = None
                elif(rand_num<.3):    # If batter hits ground ball, out made at home, runner on second advances
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.33):    # If batter hits ground ball, out made at home, runner on second stays
                    team.runners[2] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
            elif(temp_0!=None and temp_1!=None and temp_2!=None):
                rand_num = random.uniform(0,1)
                #if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                if(rand_num<0.3): 
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    batter.cum_sf+=1
                    team.runners[3]+=1
                    
                    if(rand_num<.15):
                        team.runners[2] = temp_1
                    if(rand_num<.06):
                        team.runners[1] = temp_0

                elif(rand_num<.3):    # If batter hits ground ball, out made at home, runners advance
                    team.runners[2] = temp_1
                    team.runners[1] = temp_0
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                #elif(rand_num<.4+batter.bsr/15):    # If batter hits ground ball, out made at home, runner on second stays
                elif(rand_num<0.3): 
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    team.runners[3]+=1
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})

        if(team.runners[3]>0):
            #print("adding runs", team.runners[3])
            team.runs+=team.runners[3]
            team.runners[3] = 0
            
        if(made_out):
            return 'out'

        #print("updated runners: " + str(team.runners))
        
    def print_result(self):
        """Printed score of a game
        """ 
        print("final score: " + self.away_team.city + " " + str(self.away_team.runs) + " " 
              + self.home_team.city + " " + str(self.home_team.runs))     
    
    def print_record(self):
        """Printed record of teams
        """ 
        print(str(self.away_team.wins) + "-" + str(self.away_team.losses))
        print(str(self.home_team.wins) + "-" + str(self.home_team.losses))     
   
    def play(self):
        """A game between two instances of the team class is played
        """ 
        self.away_team.restart()
        self.away_team.game_no+=1
        self.away_team.rotation_index = (self.away_team.game_no-1)%5
        self.home_team.restart() 
        self.home_team.game_no+=1
        self.home_team.rotation_index = (self.home_team.game_no-1)%5
        inning = 1
        game_over = False
        prin = False
        
         
        while(inning<self.innings+1 or self.away_team.runs==self.home_team.runs):
            for side in [self.away_team, self.home_team]:
                if side==self.away_team:
                    batting_team='Away'
                else:
                    batting_team = 'Home'
                    
                if(inning==self.innings and side==self.home_team and self.home_team.runs>self.away_team.runs):
                    game_over = True
                    break
            
                if prin:
                    print('inning: ' + str(inning))    
                outs = 0

                if(inning==1 and side==self.away_team):
                    # Determining starting pitchers
                    home_pitcher = self.home_team.starter
                    away_pitcher = self.away_team.starter
                    self.home_team.starter.cum_G+=1
                    self.home_team.starter.cum_GS+=1
                    self.away_team.starter.cum_G+=1
                    self.away_team.starter.cum_GS+=1
                    self.home_team.current_pitcher_list.append(home_pitcher)
                    self.home_team.current_pitcher = home_pitcher
                    self.away_team.current_pitcher_list.append(away_pitcher)
                    self.away_team.current_pitcher = away_pitcher
                    #self.away_team.lineup, self.away_team.bench = self.away_team.make_lineup(self.away_team.batters, self.home_team.current_pitcher, self.home_team.league, self.away_team.lineup_dict)
                    #self.home_team.lineup, self.home_team.bench = self.home_team.make_lineup(self.home_team.batters, self.away_team.current_pitcher, self.home_team.league, self.home_team.lineup_dict)
                    

                while(outs<3):
                    # Choosing a pitcher for a team based on game factors
                    if(side==self.away_team):
                        current_pitcher = self.home_team.current_pitcher
                        if prin:
                            print("pitcher", current_pitcher.name)

                        pitcher = self.home_team.choose_pitcher(outs, inning, self.innings, side.runs, self.home_team.runs, side.lineup, 
                                                                side.batting_index, self.home_team.lineup, self.home_team.batting_index)
                        
                        if(current_pitcher!=pitcher and self.home_team.lineup[8].name=='pitcher'):   # If starting pitcher gets taken out of game and lineup, put bench player in to bat
                            self.home_team.lineup[8] = self.home_team.bench[random.randint(0, len(self.home_team.bench)-1)]
                            self.home_team.lineup[8].cum_G+=1
                            
                    elif(side==self.home_team):
                        current_pitcher = self.away_team.current_pitcher
                        if prin:
                            print("pitcher", current_pitcher.name)
                        pitcher = self.away_team.choose_pitcher(outs, inning, self.innings, side.runs, self.away_team.runs, side.lineup, 
                                                                side.batting_index, self.away_team.lineup, self.away_team.batting_index)
                        
                        if(current_pitcher!=pitcher and self.away_team.lineup[8].name=='pitcher'):   # If starting pitcher gets taken out of game and lineup, put bench player in to bat
                            self.away_team.lineup[8] = self.away_team.bench[random.randint(0, len(self.away_team.bench)-1)]
                            self.away_team.lineup[8].cum_G+=1
                            
                    # Determining whether stolen base should be attempted
                    stolen_base_result = self.get_sb_wp_result(side.runners, pitcher, side, outs, random.uniform(0,1))
                    
                    if(stolen_base_result=='out'):   # If failed stolen base, out is recorded
                        pitcher.current_outs+=1
                        outs+=1
                                
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.runners_dict = {}
                            break
                    
                    # Determining outcome of at-bat
                    if prin:
                        print("batter", side.lineup[side.batting_index].name)
                    rand_num = random.uniform(0,2)
                    result = self.get_result(side.lineup[side.batting_index], pitcher, outs, side.runners, batting_team, rand_num)
                    
                    if prin:
                        print("result: " + result)
                    if(result=='out' or result=='k' or result=='sf'):    # Batter made an out
                        pitcher.current_outs+=1
                        outs+=1
                        if prin:
                            print("outs: " + str(outs))
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.batting_index+=1    # Gettting next batter in lineup
                            side.batting_index = side.batting_index%9
                            side.runners_dict = {}
                            break
                        else:
                            self.move_runners(side, side.lineup[side.batting_index], pitcher, outs, result)
                            
                    elif(result=='gdp'):    # Batter made an out
                        pitcher.current_outs+=2
                        outs+=2
                        if prin:
                            print("outs: " + str(outs))
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.batting_index+=1    # Gettting next batter in lineup
                            side.batting_index = side.batting_index%9
                            side.runners_dict = {}
                            break
                        else:
                            self.move_runners(side, side.lineup[side.batting_index], pitcher, outs, result)
                            
                    else:    # Batter reached, runners are moved
                        running_result = self.move_runners(side, side.lineup[side.batting_index], pitcher, outs, result)
                        if(running_result=='out'):    # Baserunner made an out
                            pitcher.current_outs+=1
                            outs+=1
                            if prin:
                                print("outs: " + str(outs))
                            
                            if(outs>2):   # If outs = 3, end of inning
                                pitcher.finished_half = True
                                side.batting_index+=1    # Gettting next batter in lineup
                                side.batting_index = side.batting_index%9
                                side.runners_dict = {}
                                break
                        if prin:
                            print(side.city + ': '+ str(side.runs))
                        if(inning>9 and self.away_team.runs<self.home_team.runs):
                            game_over = True
                    
                    if(game_over):    # If game is over, escape game loop
                        break
                        
                    side.batting_index+=1    # Gettting next batter in lineup
                    side.batting_index = side.batting_index%9
                    
                if(game_over):    # If game is over, escape game loop
                    break
                        
                side.reset_runners()    # Runners are reset after half inning
            

            if(game_over):    # If game is over, escape game loop
                break   
                
            inning+=1
        
        # After game, updating team records and number of cumulative runs
        
        if(self.away_team.runs>self.home_team.runs):
            self.away_team.wins+=1
            self.home_team.losses+=1
        elif(self.away_team.runs<self.home_team.runs):
            self.away_team.losses+=1
            self.home_team.wins+=1
        
        self.away_team.cum_runs+=self.away_team.runs
        self.away_team.cum_runs_allowed+=self.home_team.runs
        self.home_team.cum_runs+=self.home_team.runs
        self.home_team.cum_runs_allowed+=self.away_team.runs             
               
class Season_new(Game_new):
    """Class that holds information for a Season of games.
    
    Attributes:

    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class
        
        Args:
            away_team: away team name
            home_team: home team name
        """
        self.away_team = away_team
        self.home_team = home_team

    def simulate_season(self, games):
        """A season of games is simulated
        
        Args:
            games: number of games to be played
        """
        for game in range(1, games+1):
            g = Game(self.away_team, self.home_team)

            g.play() 
            
    def reset_season(self, teams):
        """Reinitializing teams
        
        Args:
            teams, Team instances to reinitialize
        """
        for team in teams:
            team.__init__(team.city, team.league, team.batter_df, team.pitcher_df, team.starters, team.relievers_df)

In [147]:
def convert_string(row, column):
    
    if(column in row.index):
        if(row[column]!=''):
            return(str(row[column]))
        else:
            return('0')

def convert_int(row, column):
    
    if(column in row.index):
         if(row[column]!=''):
            return(int(row[column]))
         else:
            return(0)
        
def convert_float(row, column):
    
    if(column in row.index):
        if(row[column]!=''):
            return(float(row[column]))
        else:
            return(0.0)
             
def reading_data(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    return(pd.read_csv(file_name))
  
def writing_data(df, filename):
    """Writing to spreadsheet
    Args:
        df: data
        file_name: path of file
    """
    df.to_csv(filename, index=False)
    
def column_type(stats_df, batter):
    """Fixing column types of read in csv files
    Args:
        stats_df: dataframe of stats
        batter: True if working with batter statistics
    Returns:
        stats_df: Updated_stats
    """
    if(batter):
        string_columns = ['Season', 'Name', 'Bats', 'Tm', 'playerId']
        int_columns = ['G_R', 'PA_R', 'AB_R',
                       'H_R', '1B_R', '2B_R', '3B_R', 'HR_R', 'R_R', 'RBI_R', 'BB_R', 'IBB_R','SO_R', 'HBP_R', 'SF_R',
                       'SH_R', 'GDP_R', 'SB_R', 'CS_R', 'G_L', 'PA_L', 'AB_L', 'H_L', '1B_L', '2B_L', '3B_L', 'HR_L', 
                       'R_L', 'RBI_L', 'BB_L', 'IBB_L', 'SO_L', 'HBP_L', 'SF_L', 'SH_L', 'GDP_L', 'SB_L', 'CS_L']
        float_columns = ['AVG_R', 'AVG_L']
    else:
        string_columns = ['Season', 'Name', 'Throws', 'Team', 'playerId']
        int_columns = ['IP_L', 'TBF_L', 'H_L', '1B_L', '2B_L', '3B_L', 'R_L', 'ER_L', 'HR_L', 'BB_L', 'IBB_L', 
                       'HBP_L','SO_L', 'IP_R', 'TBF_R', 'H_R', '2B_R', '3B_R', 'R_R', 'ER_R', 'HR_R', 'BB_R', 'IBB_R', 
                       'HBP_R', 'SO_R']
        float_columns = ['ERA_L', 'AVG_L', 'OBP_L', 'SLG_L', 'wOBA_L', 'ERA_R', 'AVG_R', 'OBP_R', 'SLG_R', 'wOBA_R']

        for column in stats_df.columns:
            if(column in string_columns):
                stats_df[column] = stats_df.apply(lambda row: convert_string(row, column), axis=1)
            elif(column in int_columns):
                stats_df[column] = stats_df.apply(lambda row: convert_float(row, column), axis=1)
            elif(column in float_columns):
                stats_df[column] = stats_df.apply(lambda row: convert_float(row, column), axis=1)
   
            
    return stats_df
    
def date_formatter(row):
    """Formats dates
    Args:
        row: row of data
    Returns:
        date in proper form('mm/dd/yyyy')
    """
    if row.date == '':
        return(row.date)
    else:
        split_date = row.date.split('-')
        day = split_date[0]
        day = day.zfill(2)
        month = strptime(split_date[1],'%b').tm_mon
        date = str(month) +  '/' + day + '/' + '2020'
        date = datetime.datetime.strptime(date, '%m/%d/%Y')
        date = date.strftime('%m/%d/%Y')
        
    return date
    
def unknown_player_id(id_arm_dict, new_player_id, pitchers=False):
    """Finds batting or pitching side of players who do not have data in original stats dataframes
    
    Args:
        id_arm_dict: dict with player ID's as keys and arm side as values
        new_player_id: dict with player names as keys and their ID's as values
        pitchers: True if dict is of pitchers. Default=False
    Returns:    
        id_arm_dict: dict with ID's as keys and arm or batting side as values
    """

    p_dict = {}
    url_dict = {}
    url_start = 'https://www.fangraphs.com/players/'
    pos_modifier = 0
    
    if(pitchers):
        pos_modifier = 2
        
    for k, v in new_player_id.items():
        if(k[-1]=='.'):
            k = k[:-1]
        k = k.replace('\xa0', '')
        k = k.replace(' ', '-')
        p_dict.update({k: v})
        url_dict.update({v: url_start+k+'/'+v})


    for ID, url in url_dict.items():
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')

        for tag in soup.findAll('div', attrs={'class': 'player-info-box-item'}):

            if(tag.get_text()!='' and tag.get_text().split()[0]=='Bats/Throws:'):
                hand = tag.get_text().split(' ')[1][pos_modifier]
                if(hand=='B'):
                    hand = 'S'
                id_arm_dict.update({ID: hand})

    return id_arm_dict

def batter_stats_espn():

    url = 'http://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2019/start/'
    links = [url+str(i) for i in range(1, 350, 50)]
    headers = {'User-Agent': header_name}

    batter_stats = pd.DataFrame()

    for (num, link) in enumerate(links):
        source = requests.get(link, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        table = soup.find('table', attrs={'class': 'tablehead'})
        if(num==0):
            column_headers = []
            table_headers = table.find('tr', attrs={'class': 'colhead'})
            for header_value in table_headers.findAll('td'):
                column_headers.append(header_value.get_text())

            column_headers[0] = 'ID'
            batter_stats = pd.DataFrame(columns=column_headers)

        df = pd.DataFrame(columns=column_headers)            


        for player in table.find_all('tr', attrs={'class': re.compile('row player-10-')}):
            player_values = []
            player_values.append(player['class'][1].split('-')[2])

            values =  player.findAll('td')
            values.pop(0)
            for value in values:
                player_values.append(value.get_text())

            df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

        batter_stats = pd.concat([batter_stats, df], ignore_index=True)

    return batter_stats

def pitcher_stats_espn():
    url = 'http://www.espn.com/mlb/history/leaders/_/type/pitching/breakdown/season/year/2019/sort/wins/start/'
    links = [url+str(i) for i in range(1, 500, 50)]
    headers = {'User-Agent': header_name}

    pitcher_stats = pd.DataFrame()

    for (num, link) in enumerate(links):
        source = requests.get(link, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        table = soup.find('table', attrs={'class': 'tablehead'})
        if(num==0):
            column_headers = []
            table_headers = table.find('tr', attrs={'class': 'colhead'})
            for header_value in table_headers.findAll('td'):
                column_headers.append(header_value.get_text())

            column_headers[0] = 'ID'
            pitcher_stats = pd.DataFrame(columns=column_headers)

        df = pd.DataFrame(columns=column_headers)            


        for player in table.find_all('tr', attrs={'class': re.compile('row player-10-')}):
            player_values = []
            player_values.append(player['class'][1].split('-')[2])

            values =  player.findAll('td')
            values.pop(0)
            for value in values:
                player_values.append(value.get_text())

            df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

        pitcher_stats = pd.concat([pitcher_stats, df], ignore_index=True)

    return pitcher_stats

def create_player_id_dict(batter_stats, pitcher_stats):
    """Creates Team Instances with dataframes of player values of a given team
        
        Args:
            batter_stats: df of batter stats
            pitcher_stats: df of pitcher stats
        Returns:
            team_dict: dict with city of team as key and Team instance as value
        """
    url_list = []
    url_start = 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid='
    team_dict = {}
    new_batter_id = {}
    new_pitcher_id = {}

    for num in range(1,31):
        url_list.append(url_start+str(num))
    url_list = ['https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid=16', 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid=9']
    for url in url_list:
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        positional_values_dict = {}
        pitcher_df = pd.DataFrame()
        batter_df = pd.DataFrame()

        for a_tag in soup.findAll('a', attrs={'href': '#C'}):
            team_name = a_tag.find_next('span').get_text()
            team_name = team_name.replace('\xa0', '')

        for a_tag in soup.findAll('a', attrs={'href': ['#C', '#1B', '#2B', '#SS', '#3B', '#LF', '#CF', '#RF', '#DH', '#SP', '#RP', '#ALL']}):
            pos = a_tag.get_text()

            if(pos in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH', 'ALL Batters']):
                column_names = ['Name', 'ID', 'PA', 'AVG', 'OBP', 'SLG', 'wOBA', 'Bat', 'BsR', 'Fld', 'WAR']
            else:
                column_names = ['Name', 'ID', 'IP', 'K/9', 'BB/9', 'HR/9', 'BABIP', 'LOB', 'ERA', 'FIP', 'WAR']

            pos_df = pd.DataFrame(columns=column_names)

            table = a_tag.find_next('table')
            

            for row in table.findAll('tr', attrs={'class': 'depth_reg'}):
                info = row.a['href']
                elements = info.split('=')
                player_ID = elements[1].split('&')[0]
                values_list = []

                for value in row.findAll('td'):
                    values_list.append(value.get_text())

                    if(len(values_list)==1):
                        values_list.append(player_ID)

                pos_df = pos_df.append(pd.Series(values_list, index=pos_df.columns), ignore_index=True)   

            positional_values_dict.update({pos: pos_df})   
        print(positional_values_dict)
        batter_df = pd.DataFrame()    
        pos_dict = {}

        for pos, df in positional_values_dict.items():
            if(pos in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH']):
                for index, row in df.iterrows():
                    if(row['ID'] in batter_stats['playerId'].values.tolist()):
                        if(not batter_df.empty and row['ID'] in batter_df['playerId'].values.tolist()):
                            continue
                        else:
                            continue
                    else:
                        if(not batter_df.empty and row['ID'] in batter_df['playerId'].values.tolist()):
                            continue
                        else:
                            new_batter_id.update({row['Name']: row['ID']})
                            print(row['Name']+ '  ' + pos)

                            
            elif(pos in ['SP', 'RP']):
                for index, row in df.iterrows():
                    if(row['ID'] in pitcher_stats['playerId'].values.tolist()):
                        if(not pitcher_df.empty and row['ID'] in pitcher_df['playerId'].values.tolist()):
                            continue
                        else:
                            continue
                    else:
                        if(not pitcher_df.empty and row['ID'] in pitcher_df['playerId'].values.tolist()):
                            continue
                        else:
                            new_pitcher_id.update({row['Name']: row['ID']})
                            print(row['Name']+ '  ' + pos)

    return new_batter_id, new_pitcher_id         

def daily_lineups():
    url = 'https://www.mlb.com/starting-lineups'
    headers = {'User-Agent': header_name}
    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    lineup_dict = {}
    schedule_dict = {}
    schedule_dict[datetime.datetime.now().strftime('%Y-%m-%d')] = []
    for matchup in soup.findAll('div', attrs={'class': 'starting-lineups__matchup'}):
        for num, team_names in enumerate(matchup.findAll('a', attrs={'class': 'starting-lineups__team-name--link'})): # Getting team names
            if(num==0):
                away_team = nickname_to_city_dict[team_names.get_text().strip().lower()]
                
                if(away_team in lineup_dict.keys()):
                    away_team = away_team + '2'
            elif(num==1):
                home_team = nickname_to_city_dict[team_names.get_text().strip().lower()]
                
                if(home_team in lineup_dict.keys()):
                    home_team = home_team + '2'

        # Don't get lineup info if game is postponed        
        if(matchup.find('span', attrs={'class': 'starting-lineups__game-state starting-lineups__game-state--postponed'})):
            break

        schedule_dict[datetime.datetime.now().strftime('%Y-%m-%d')].append((away_team, home_team))  # Adding matchup to schedule dict

        for num, names in enumerate(matchup.findAll('div', attrs={'class': 'starting-lineups__pitcher-name'})): # Getting starter names
            for pitcher_names in names.findAll('a', attrs={'class': 'starting-lineups__pitcher--link'}):
                if(num==0):
                    away_starter = pitcher_names.get_text()
                elif(num==1):
                    home_starter = pitcher_names.get_text()


        away_lineup = {}
        home_lineup = {}

        # Gathering team lineups and putting them in a dict
        for lineups in matchup.findAll('div', attrs={'class': 'starting-lineups__teams starting-lineups__teams--sm starting-lineups__teams--xl'}):
            for a_lineup in lineups.findAll('ol', attrs={'class': 'starting-lineups__team starting-lineups__team--away'}):
                for batter in a_lineup.findAll('li'):
                    if(batter.get_text()=='TBD'):
                        break
                    away_lineup[batter.a.get_text()] = batter.span.get_text().split()[1]
                
                if(len(away_starter)==0):  # Setting starter to None if no starter found
                    away_starter = None
                if(len(away_lineup)==0):  # Setting lineup to None if no Lineup found
                    away_lineup = None
                    
                lineup_dict[away_team] = {'starter': away_starter, 'lineup': away_lineup}
                
            for h_lineup in lineups.findAll('ol', attrs={'class': 'starting-lineups__team starting-lineups__team--home'}):
                for batter in h_lineup.findAll('li'):
                    if(batter.get_text()=='TBD'):
                        break
                    home_lineup[batter.a.get_text()] = batter.span.get_text().split()[1]

                if(len(home_starter)==0):  # Setting starter to None if no starter found
                    home_starter = None
                if(len(home_lineup)==0):  # Setting lineup to None if no Lineup found
                    home_lineup = None
                    
                lineup_dict[home_team] = {'starter': home_starter, 'lineup': home_lineup}
        
    return lineup_dict, schedule_dict

def fangraphs_daily_lineups(lineup_dict={}):
    url = 'https://www.fangraphs.com/livescoreboard.aspx?'
    headers = {'User-Agent': header_name}
    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    

    schedule_dict = {}
    player_dict = {}
    schedule_dict[datetime.datetime.now().strftime('%m/%d/%Y')] = []

    header = soup.find('tr', attrs={'class': 'win-board__header'})

    for games in header.next_siblings:
        for matchup in games.findAll('td', style=True, align=False):
            away_team = None
            home_team = None
            
            for num, team_names in enumerate(matchup.findAll('a')): # Getting team names

                if(num==0):
                    if team_names.get_text().strip().capitalize() in league.keys():
                        away_team = team_names.get_text().strip().capitalize()
                    else:
                        away_team = nickname_to_city_dict[team_names.get_text().strip().lower()]
                    
                    for teams in schedule_dict[datetime.datetime.now().strftime('%m/%d/%Y')]:                        
                        if away_team in teams:                    
                            away_team = away_team + '2'
                elif(num==1):

                    if team_names.get_text().strip().capitalize() in league.keys():
                        home_team = team_names.get_text().strip().capitalize()
                    else:
                        home_team = nickname_to_city_dict[team_names.get_text().strip().lower()]

                    for teams in schedule_dict[datetime.datetime.now().strftime('%m/%d/%Y')]:                        
                        if home_team in teams:  
                            home_team = home_team + '2'

                        
            if(away_team!=None and home_team!=None):   # Adding matchup to schedule dict
                schedule_dict[datetime.datetime.now().strftime('%m/%d/%Y')].append((away_team, home_team))            
            
            
            away_lineup_dict = {}
            away_lineup = []
            away_starter = None
            away_opener = None
            home_lineup_dict = {}
            home_lineup = []
            home_starter = None
            home_opener = None

            lineup = matchup.find('table', attrs={'class', 'lineup'})
            if(lineup!=None):
                for num, position in enumerate(lineup.findAll('td', attrs={'align': 'left'})):

                    if(num%2==0):
                        if position.get_text()[0:2] == 'SP':
                            away_starter = position.a.get_text().strip()
                            away_starter_id = position.a['href'].split('playerid=')[1].split('&')[0]
                            player_dict[away_starter] = away_starter_id
                        elif position.get_text()[0:2] == 'OP':
                            away_opener = position.a.get_text().strip()
                            away_opener_id = position.a['href'].split('playerid=')[1].split('&')[0]
                            player_dict[away_opener] = away_opener_id
                        elif position.get_text()[0:2] not in ('SP', 'PP', 'OP'):
                            for player in position.findAll('a'):
                                away_batter = player.get_text().strip()
                                away_lineup.append(away_batter)
                                away_batter_id = position.a['href'].split('playerid=')[1].split('&')[0]
                                player_dict[away_batter] = away_batter_id

                        away_lineup_dict['starter'] = away_starter
                        away_lineup_dict['opener'] = away_opener
                        away_lineup_dict['lineup'] = away_lineup

                    elif(num%2==1):
                        if position.get_text()[0:2] == 'SP':
                            home_starter = position.a.get_text().strip()
                            home_starter_id = position.a['href'].split('playerid=')[1].split('&')[0]
                            player_dict[home_starter] = home_starter_id
                        elif position.get_text()[0:2] == 'OP':
                            home_opener = position.a.get_text().strip()
                            home_opener_id = position.a['href'].split('playerid=')[1].split('&')[0]
                            player_dict[home_opener] = home_opener_id
                        elif position.get_text()[0:2] not in ('SP', 'PP', 'OP'):
                            for player in position.findAll('a'):
                                home_batter = player.get_text().strip()
                                home_lineup.append(home_batter)
                                home_batter_id = position.a['href'].split('playerid=')[1].split('&')[0]
                                player_dict[home_batter] = home_batter_id

                            home_lineup_dict['starter'] = home_starter
                            home_lineup_dict['opener'] = home_opener
                            home_lineup_dict['lineup'] = home_lineup    

                  
                #if((len(lineup_dict[away_team]['lineup'])>0 or len(away_lineup_dict['lineup']>0)     # Only update for games where lineups exist
                    #and (len(lineup_dict[away_team]['lineup'])>0 or len(away_lineup_dict['lineup']>0))):
                            
                #if lineup already exists and new lineup is empty, do not replace existing lineup
                if not (away_team in lineup_dict.keys() and len(lineup_dict[away_team]['lineup'])>0  and len(away_lineup_dict['lineup'])==0):   
                    lineup_dict[away_team] = away_lineup_dict

                if not (home_team in lineup_dict.keys() and len(lineup_dict[home_team]['lineup'])>0  and len(home_lineup_dict['lineup'])==0):    
                    lineup_dict[home_team] = home_lineup_dict
        
    return lineup_dict, schedule_dict, player_dict

def simulate_slate_new(lineup_dict, rosters_dict, schedule_dict, num_games=400):
    
    for team, lineup in lineup_dict.items():        
        team.__init__(team, league[team], lineup, rosters_dict[team])
    
    win_rate = pd.Series()
    for matchup in schedule_dict[datetime.datetime.now().strftime('%Y-%m-%d')]:
        away_team = matchup[0]
        home_team = matchup[1]
        
        for num in range(num_games):
            g = Game(team_dict[away_team], team_dict[home_team])
            g.play()
            g.print_result()

        win_rate = win_rate.append(pd.Series([team_dict[away_team].wins/num_games], index=[away_team]))
        win_rate = win_rate.append(pd.Series([team_dict[home_team].wins/num_games], index=[home_team]))
    
    
    return win_rate

def create_teams_new(batter_stats, pitcher_stats, lineup_dict={}, rosters={}):
    """Creates Team Instances with dataframes of player values of a given team
        
        Args:
            batter_stats: df of batter stats
            pitcher_stats: df of pitcher stats
            lineup_dict: dict of team lineups and starting pitchers
            rosters: active rosters of players for each team
        Returns:
            team_dict: dict with city of team as key and Team instance as value
    """
    global id_arms_dict, id_bats_dict, batter_stats_df, pitcher_stats_df, team_num_dict
    
    team_dict = {}
    url_list = []
    url_start = 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid='


    for team_name in lineup_dict.keys():    # Get list of url's for teams playing
        #url_list.append(url_start+str(team_num_dict[name])) 
        
        name = team_name.split('2')[0]
        
        if team_name[-1]=='2':   # If team is from second leg of double header, no need to update roster information
            team_dict.update({team_name: Team_new(name, league[name], batter_df, pitcher_df, rosters[name], 
                                                  batter_stats, pitcher_stats, lineup_dict[name])})
            continue
        
        
        url = url_start+str(team_num_dict[name])
    #url_list = ['https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid=5', 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid=10']
    
    #for url in url_list:    # Getting players on each team
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        positional_values_dict = {}
        pitcher_df = pd.DataFrame()
        batter_df = pd.DataFrame()
        pos_dict = {}

        #for a_tag in soup.findAll('a', attrs={'href': '#C'}):
            #team_name = a_tag.find_next('span').get_text().strip()

            #if team_name not in league.keys() and team_name.capitalize() not in league.keys():
                #team_name = nickname_to_city_dict[team_name.replace('\xa0', '').lower()]
            
        for a_tag in soup.findAll('a', attrs={'href': ['#C', '#1B', '#2B', '#SS', '#3B', '#LF', '#CF', '#RF', '#DH', '#SP', '#RP']}):#, '#ALL']}):
            pos = a_tag.get_text()
            pos_dict[pos] = {}
            rosters[team_name][pos] = []

            table = a_tag.find_next('table')

            for row in table.findAll('tr', attrs={'class': 'depth_reg'}):
                if not row.a:
                    continue
                    
                if row.span:   # Players on dl
                    continue
                    
                info = row.a['href']
                elements = info.split('=')
                player_ID = elements[1].split('&')[0]

                if not player_ID.isdigit():
                    continue

                player_name = unidecode.unidecode(row.a.get_text().strip())   # Removing accent marks
                pos_dict[pos][player_ID] = player_name
                rosters[team_name][pos].append(player_name)

        
        for position, player_info in pos_dict.items():
            for player_ID, player_name in player_info.items():
                
                if position not in rosters[team_name].keys() or player_name not in rosters[team_name][position]:
                    continue

                if position in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH']:

                    if(not batter_df.empty and player_ID in batter_df['playerID'].values.tolist()):  # player already in batter_df at different position
                        batter_df.loc[batter_df['playerID']==player_ID, position] = 1
                    else:

                        batter_headers = ['Name', 'playerID', 'Team', 'Bats', 'PA_L', 'AB_L',
                                           'singles_L', 'doubles_L', 'triples_L', 'HR_L', 
                                           'BB_L', 'SO_L', 'HBP_L', 'SB_L', 'CS_L', 
                                           'PA_R', 'AB_R', 'singles_R', 'doubles_R', 
                                           'triples_R', 'HR_R', 'BB_R', 'SO_R', 'HBP_R', 
                                           'SB_R', 'CS_R', 'PA_Home', 'AB_Home',
                                           'singles_Home', 'doubles_Home', 'triples_Home', 'HR_Home', 
                                           'BB_Home', 'SO_Home', 'HBP_Home', 'SB_Home', 'CS_Home', 
                                           'PA_Away', 'AB_Away', 'singles_Away', 'doubles_Away', 
                                           'triples_Away', 'HR_Away', 'BB_Away', 'SO_Away', 'HBP_Away', 
                                           'SB_Away', 'CS_Away', 'C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH', 'SP', 'RP']

                        player_stats = batter_stats.loc[batter_stats['playerId']==int(player_ID)]

                        if player_stats.empty:  # Change this after finding handedness of all players
                            continue

                        if(player_stats['Bats'].tolist()[0]=='L'):
                            PA_L = 50
                            AB_L = 45
                            singles_L = 7
                            doubles_L = 2
                            triples_L = 0
                            HR_L = 2
                            BB_L = 5
                            HBP_L = 0
                            SO_L = 12
                            SB_L = 1
                            CS_L = 0

                            PA_R = 80
                            AB_R = 72
                            singles_R = 12
                            doubles_R = 4
                            triples_R = 0
                            HR_R = 4
                            BB_R = 7
                            HBP_R = 1
                            SO_R = 18
                            SB_R = 1
                            CS_R = 0
                        elif(player_stats['Bats'].tolist()[0]=='R'):
                            PA_L = 50
                            AB_L = 45
                            singles_L = 8
                            doubles_L = 2
                            triples_L = 0
                            HR_L = 3
                            BB_L = 5
                            HBP_L = 0
                            SO_L = 10
                            SB_L = 1
                            CS_L = 0

                            PA_R = 80
                            AB_R = 72
                            singles_R = 10
                            doubles_R = 4
                            triples_R = 0
                            HR_R = 3
                            BB_R = 7
                            HBP_R = 1
                            SO_R = 20
                            SB_R = 1
                            CS_R = 0
                        else:
                            PA_L = 50
                            AB_L = 45
                            singles_L = 8
                            doubles_L = 2
                            triples_L = 0
                            HR_L = 3
                            BB_L = 5
                            HBP_L = 0
                            SO_L = 10
                            SB_L = 1
                            CS_L = 0

                            PA_R = 80
                            AB_R = 72
                            singles_R = 12
                            doubles_R = 4
                            triples_R = 0
                            HR_R = 4
                            BB_R = 7
                            HBP_R = 1
                            SO_R = 18
                            SB_R = 1
                            CS_R = 0

                        PA_Home = 80
                        AB_Home = 72
                        singles_Home = 12
                        doubles_Home = 4
                        triples_Home = 0
                        HR_Home = 4
                        BB_Home = 7
                        HBP_Home = 1
                        SO_Home = 18
                        SB_Home = 1
                        CS_Home = 0

                        PA_Away = 80
                        AB_Away = 72
                        singles_Away = 12
                        doubles_Away = 4
                        triples_Away = 0
                        HR_Away = 4
                        BB_Away = 7
                        HBP_Away = 1
                        SO_Away = 18
                        SB_Away = 1
                        CS_Away = 0

                        seasons_list = sorted(player_stats['Season'].to_list())
                        for year in seasons_list:
                            multiplier = (1-(max(seasons_list)-year)/len(seasons_list))

                            PA_L += player_stats.loc[player_stats['Season']==year]['PA_L'].values[0]*multiplier
                            AB_L += player_stats.loc[player_stats['Season']==year]['AB_L'].values[0]*multiplier
                            singles_L += player_stats.loc[player_stats['Season']==year]['singles_L'].values[0]*multiplier
                            doubles_L += player_stats.loc[player_stats['Season']==year]['doubles_L'].values[0]*multiplier
                            triples_L += player_stats.loc[player_stats['Season']==year]['triples_L'].values[0]*multiplier
                            HR_L += player_stats.loc[player_stats['Season']==year]['HR_L'].values[0]*multiplier
                            BB_L += player_stats.loc[player_stats['Season']==year]['BB_L'].values[0]*multiplier
                            HBP_L += player_stats.loc[player_stats['Season']==year]['HBP_L'].values[0]*multiplier
                            SO_L += player_stats.loc[player_stats['Season']==year]['SO_L'].values[0]*multiplier
                            #SB_L += player_stats.loc[player_stats['Season']==year]['SB_L'].values[0]*multiplier
                            #CS_L += player_stats.loc[player_stats['Season']==year]['CS_L'].values[0]*multiplier

                            PA_R += player_stats.loc[player_stats['Season']==year]['PA_R'].values[0]*multiplier
                            AB_R += player_stats.loc[player_stats['Season']==year]['AB_R'].values[0]*multiplier
                            singles_R += player_stats.loc[player_stats['Season']==year]['singles_R'].values[0]*multiplier
                            doubles_R += player_stats.loc[player_stats['Season']==year]['doubles_R'].values[0]*multiplier
                            triples_R += player_stats.loc[player_stats['Season']==year]['triples_R'].values[0]*multiplier
                            HR_R += player_stats.loc[player_stats['Season']==year]['HR_R'].values[0]*multiplier
                            BB_R += player_stats.loc[player_stats['Season']==year]['BB_R'].values[0]*multiplier
                            HBP_R += player_stats.loc[player_stats['Season']==year]['HBP_R'].values[0]*multiplier
                            SO_R += player_stats.loc[player_stats['Season']==year]['SO_R'].values[0]*multiplier
                            #SB_R += player_stats.loc[player_stats['Season']==year]['SB_R'].values[0]*multiplier
                            #CS_R += player_stats.loc[player_stats['Season']==year]['CS_R'].values[0]*multiplier

                            PA_Home += player_stats.loc[player_stats['Season']==year]['PA_Home'].values[0]*multiplier
                            AB_Home += player_stats.loc[player_stats['Season']==year]['AB_Home'].values[0]*multiplier
                            singles_Home += player_stats.loc[player_stats['Season']==year]['singles_Home'].values[0]*multiplier
                            doubles_Home += player_stats.loc[player_stats['Season']==year]['doubles_Home'].values[0]*multiplier
                            triples_Home += player_stats.loc[player_stats['Season']==year]['triples_Home'].values[0]*multiplier
                            HR_Home += player_stats.loc[player_stats['Season']==year]['HR_Home'].values[0]*multiplier
                            BB_Home += player_stats.loc[player_stats['Season']==year]['BB_Home'].values[0]*multiplier
                            HBP_Home += player_stats.loc[player_stats['Season']==year]['HBP_Home'].values[0]*multiplier
                            SO_Home += player_stats.loc[player_stats['Season']==year]['SO_Home'].values[0]*multiplier
                            #SB_Home += player_stats.loc[player_stats['Season']==year]['SB_Home'].values[0]*multiplier
                            #CS_Home += player_stats.loc[player_stats['Season']==year]['CS_Home'].values[0]*multiplier

                            PA_Away += player_stats.loc[player_stats['Season']==year]['PA_Away'].values[0]*multiplier
                            AB_Away += player_stats.loc[player_stats['Season']==year]['AB_Away'].values[0]*multiplier
                            singles_Away += player_stats.loc[player_stats['Season']==year]['singles_Away'].values[0]*multiplier
                            doubles_Away += player_stats.loc[player_stats['Season']==year]['doubles_Away'].values[0]*multiplier
                            triples_Away += player_stats.loc[player_stats['Season']==year]['triples_Away'].values[0]*multiplier
                            HR_Away += player_stats.loc[player_stats['Season']==year]['HR_Away'].values[0]*multiplier
                            BB_Away += player_stats.loc[player_stats['Season']==year]['BB_Away'].values[0]*multiplier
                            HBP_Away += player_stats.loc[player_stats['Season']==year]['HBP_Away'].values[0]*multiplier
                            SO_Away += player_stats.loc[player_stats['Season']==year]['SO_Away'].values[0]*multiplier
                            #SB_Away += player_stats.loc[player_stats['Season']==year]['SB_Away'].values[0]*multiplier
                            #CS_Away += player_stats.loc[player_stats['Season']==year]['CS_Away'].values[0]*multiplier


                        stats = pd.Series({'Name': player_name, 'playerID': player_ID, 'Team': team_name, 
                                           'Bats': player_stats['Bats'].tolist()[0], 'PA_L': PA_L, 'AB_L': AB_L,
                                           'singles_L': singles_L, 'doubles_L': doubles_L, 'triples_L': triples_L, 'HR_L': HR_L, 
                                           'BB_L': BB_L, 'SO_L': SO_L, 'HBP_L': HBP_L, 'SB_L': SB_L, 'CS_L': CS_L, 
                                           'PA_R': PA_R, 'AB_R': AB_R, 'singles_R': singles_R, 'doubles_R': doubles_R, 
                                           'triples_R': triples_R, 'HR_R': HR_R, 'BB_R': BB_R, 'SO_R': SO_R, 'HBP_R': HBP_R, 
                                           'SB_R': SB_R, 'CS_R': CS_R, 'PA_Home': PA_Home, 'AB_Home': AB_Home,
                                           'singles_Home': singles_Home, 'doubles_Home': doubles_Home, 'triples_Home': triples_Home, 'HR_Home': HR_Home, 
                                           'BB_Home': BB_Home, 'SO_Home': SO_Home, 'HBP_Home': HBP_Home, 'SB_Home': SB_Home, 'CS_Home': CS_Home, 
                                           'PA_Away': PA_Away, 'AB_Away': AB_Away, 'singles_Away': singles_Away, 'doubles_Away': doubles_Away, 
                                           'triples_Away': triples_Away, 'HR_Away': HR_Away, 'BB_Away': BB_Away, 'SO_Away': SO_Away, 'HBP_Away': HBP_Away, 
                                           'SB_Away': SB_Away, 'CS_Away': CS_Away, position: 1})



                        if(batter_df.empty):
                            batter_df = pd.DataFrame(columns=batter_headers)

                        batter_df = batter_df.append(stats, ignore_index=True)   # Getting stats from batter stats dataframe
                elif position in ['SP', 'RP']:
                    if(not pitcher_df.empty and player_ID in pitcher_df['playerID'].values.tolist()):  # player already in pitcher_df at different position
                        pitcher_df.loc[pitcher_df['playerID']==player_ID, position] = 1
                    else:

                        pitcher_headers = ['Name', 'playerID', 'Team', 'Throws', 'TBF_L',
                                           'singles_L', 'doubles_L', 'triples_L', 'HR_L', 'R_L', 
                                           'ER_L', 'BB_L','HBP_L', 'SO_L', 
                                           'TBF_R',  'singles_R', 'doubles_R', 'triples_R', 'HR_R', 'R_R', 
                                           'ER_R', 'BB_R','HBP_R', 'SO_R', 'TBF_Home',
                                           'singles_Home', 'doubles_Home', 'triples_Home', 'HR_Home', 'R_Home', 
                                           'ER_Home', 'BB_Home','HBP_Home', 'SO_Home', 
                                           'TBF_Away',  'singles_Away', 'doubles_Away', 'triples_Away', 'HR_Away', 'R_Away', 
                                           'ER_Away', 'BB_Away','HBP_Away', 'SO_Away']

                        player_stats = pitcher_stats.loc[pitcher_stats['playerId']==int(player_ID)]

                        if player_stats.empty:  # Change this after finding handedness of all players
                            continue

                        if(player_stats['Throws'].tolist()[0]=='L'):
                            TBF_L = 90
                            singles_L = 14
                            doubles_L = 4
                            triples_L = 0
                            HR_L = 2
                            R_L = 10
                            ER_L = 9
                            BB_L = 5
                            HBP_L = 0
                            SO_L = 25

                            TBF_R = 90
                            singles_R = 15
                            doubles_R = 4
                            triples_R = 0
                            HR_R = 3
                            R_R = 11
                            ER_R = 10
                            BB_R = 5
                            HBP_R = 0
                            SO_R = 20
                        elif(player_stats['Throws'].tolist()[0]=='R'):
                            TBF_L = 90
                            singles_L = 15
                            doubles_L = 4
                            triples_L = 0
                            HR_L = 3
                            R_L = 11
                            ER_L = 10
                            BB_L = 5
                            HBP_L = 0
                            SO_L = 20

                            TBF_R = 90
                            singles_R = 14
                            doubles_R = 4
                            triples_R = 0
                            HR_R = 2
                            R_R = 10
                            ER_R = 9
                            BB_R = 5
                            HBP_R = 0
                            SO_R = 25


                        TBF_Home = 90
                        singles_Home = 14
                        doubles_Home = 4
                        triples_Home = 0
                        HR_Home = 2
                        R_Home = 10
                        ER_Home = 9
                        BB_Home = 5
                        HBP_Home = 0
                        SO_Home = 25

                        TBF_Away = 90
                        singles_Away = 15
                        doubles_Away = 4
                        triples_Away = 0
                        HR_Away = 3
                        R_Away = 11
                        ER_Away = 10
                        BB_Away = 5
                        HBP_Away = 0
                        SO_Away = 20

                        seasons_list = sorted(player_stats['Season'].to_list())
                        for year in seasons_list:
                            
                            multiplier = (1-(max(seasons_list)-year)/len(seasons_list))

                            TBF_L += player_stats.loc[player_stats['Season']==year]['TBF_L'].values[0]*multiplier
                            singles_L += player_stats.loc[player_stats['Season']==year]['singles_L'].values[0]*multiplier
                            doubles_L += player_stats.loc[player_stats['Season']==year]['doubles_L'].values[0]*multiplier
                            triples_L += player_stats.loc[player_stats['Season']==year]['triples_L'].values[0]*multiplier
                            HR_L += player_stats.loc[player_stats['Season']==year]['HR_L'].values[0]*multiplier
                            R_L += player_stats.loc[player_stats['Season']==year]['R_L'].values[0]*multiplier
                            ER_L += player_stats.loc[player_stats['Season']==year]['ER_L'].values[0]*multiplier
                            BB_L += player_stats.loc[player_stats['Season']==year]['BB_L'].values[0]*multiplier
                            HBP_L += player_stats.loc[player_stats['Season']==year]['HBP_L'].values[0]*multiplier
                            SO_L += player_stats.loc[player_stats['Season']==year]['SO_L'].values[0]*multiplier


                            TBF_R += player_stats.loc[player_stats['Season']==year]['TBF_R'].values[0]*multiplier
                            singles_R += player_stats.loc[player_stats['Season']==year]['singles_R'].values[0]*multiplier
                            doubles_R += player_stats.loc[player_stats['Season']==year]['doubles_R'].values[0]*multiplier
                            triples_R += player_stats.loc[player_stats['Season']==year]['triples_R'].values[0]*multiplier
                            HR_R += player_stats.loc[player_stats['Season']==year]['HR_R'].values[0]*multiplier
                            R_R += player_stats.loc[player_stats['Season']==year]['R_R'].values[0]*multiplier
                            ER_R += player_stats.loc[player_stats['Season']==year]['ER_R'].values[0]*multiplier
                            BB_R += player_stats.loc[player_stats['Season']==year]['BB_R'].values[0]*multiplier
                            HBP_R += player_stats.loc[player_stats['Season']==year]['HBP_R'].values[0]*multiplier
                            SO_R += player_stats.loc[player_stats['Season']==year]['SO_R'].values[0]*multiplier

                            TBF_Home += player_stats.loc[player_stats['Season']==year]['TBF_Home'].values[0]*multiplier
                            singles_Home += player_stats.loc[player_stats['Season']==year]['singles_Home'].values[0]*multiplier
                            doubles_Home += player_stats.loc[player_stats['Season']==year]['doubles_Home'].values[0]*multiplier
                            triples_Home += player_stats.loc[player_stats['Season']==year]['triples_Home'].values[0]*multiplier
                            HR_Home += player_stats.loc[player_stats['Season']==year]['HR_Home'].values[0]*multiplier
                            R_Home += player_stats.loc[player_stats['Season']==year]['R_Home'].values[0]*multiplier
                            ER_Home += player_stats.loc[player_stats['Season']==year]['ER_Home'].values[0]*multiplier
                            BB_Home += player_stats.loc[player_stats['Season']==year]['BB_Home'].values[0]*multiplier
                            HBP_Home += player_stats.loc[player_stats['Season']==year]['HBP_Home'].values[0]*multiplier
                            SO_Home += player_stats.loc[player_stats['Season']==year]['SO_Home'].values[0]*multiplier


                            TBF_Away += player_stats.loc[player_stats['Season']==year]['TBF_Away'].values[0]*multiplier
                            singles_Away += player_stats.loc[player_stats['Season']==year]['singles_Away'].values[0]*multiplier
                            doubles_Away += player_stats.loc[player_stats['Season']==year]['doubles_Away'].values[0]*multiplier
                            triples_Away += player_stats.loc[player_stats['Season']==year]['triples_Away'].values[0]*multiplier
                            HR_Away += player_stats.loc[player_stats['Season']==year]['HR_Away'].values[0]*multiplier
                            R_Away += player_stats.loc[player_stats['Season']==year]['R_Away'].values[0]*multiplier
                            ER_Away += player_stats.loc[player_stats['Season']==year]['ER_Away'].values[0]*multiplier
                            BB_Away += player_stats.loc[player_stats['Season']==year]['BB_Away'].values[0]*multiplier
                            HBP_Away += player_stats.loc[player_stats['Season']==year]['HBP_Away'].values[0]*multiplier
                            SO_Away += player_stats.loc[player_stats['Season']==year]['SO_Away'].values[0]*multiplier


                        stats = pd.Series({'Name': player_name, 'playerID': player_ID, 'Team': team_name, 
                                           'Throws': player_stats['Throws'].tolist()[0], 
                                           'TBF_L': TBF_L,'singles_L': singles_L, 'doubles_L': doubles_L, 'triples_L': triples_L, 
                                           'HR_L': HR_L, 'R_L': R_L, 'ER_L': ER_L, 'BB_L': BB_L, 'HBP_L': HBP_L, 'SO_L': SO_L,  
                                           'TBF_R': TBF_R, 'singles_R': singles_R, 'doubles_R': doubles_R, 'triples_R': triples_R, 
                                           'HR_R': HR_R, 'R_R': R_R, 'ER_R': ER_R, 'BB_R': BB_R, 'HBP_R': HBP_R, 'SO_R': SO_R,
                                           'TBF_Home': TBF_Home,'singles_Home': singles_Home, 'doubles_Home': doubles_Home, 'triples_Home': triples_Home, 
                                           'HR_Home': HR_Home, 'R_Home': R_Home, 'ER_Home': ER_Home, 'BB_Home': BB_Home, 'HBP_Home': HBP_Home, 'SO_Home': SO_Home,  
                                           'TBF_Away': TBF_Away, 'singles_Away': singles_Away, 'doubles_Away': doubles_Away, 'triples_Away': triples_Away, 
                                           'HR_Away': HR_Away, 'R_Away': R_Away, 'ER_Away': ER_Away, 'BB_Away': BB_Away, 
                                           'HBP_Away': HBP_Away, 'SO_Away': SO_Away, position: 1})



                        if(pitcher_df.empty):
                            pitcher_df = pd.DataFrame(columns=pitcher_headers)

                        pitcher_df = pitcher_df.append(stats, ignore_index=True)   # Getting pitcher stats from pitcher stats dataframe
  


        batter_df.fillna(0, inplace=True)
        pitcher_df.fillna(0, inplace=True)

        team_dict.update({team_name: Team_new(team_name, league[team_name], batter_df, pitcher_df, rosters[team_name], 
                                              batter_stats, pitcher_stats, lineup_dict[team_name])})

    return team_dict, rosters

def active_rosters():
    
    headers = {'User-Agent': header_name}

    url_list = []
    url_start = 'https://www.mlb.com/'
    
    team_roster_dict = {}
    
    for team_name in mlb_teams:
        
        mlb_name = None

        # Making sure all team names are in mlb.com format
        if team_name=='red sox':
            mlb_name = 'redsox'
        elif team_name=='blue jays':
            mlb_name = 'bluejays'
        elif team_name=='d-backs':
            mlb_name = 'dbacks'
        elif team_name=='white sox':
            mlb_name = 'whitesox'
        else:
            mlb_name = team_name
     
        url = url_start+mlb_name+'/roster/depth-chart'
        position_player_dict = {}

        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')

        # Getting player names on each depth chart
        for table in soup.findAll('table', attrs={'class': 'roster__table'}):
            position = table.td.get_text()

            if position=='Rotation':
                position = 'SP'
            elif position=='Bullpen':
                position = 'RP'
            elif position=='Catcher':
                position = 'C'
            elif position=='First Base':
                position = '1B'
            elif position=='Second Base':
                position = '2B'
            elif position=='Shortstop':
                position = 'SS'
            elif position=='Third Base':
                position = '3B'
            elif position=='Left Field':
                position = 'LF'
            elif position=='Center Field':
                position = 'CF'
            elif position=='Right Field':
                position = 'RF'
            elif position=='Designated Hitter':
                position = 'DH'

            position_player_dict[position] = []

            for row in table.tbody.findAll('td', attrs={'class': 'info'}):
                if row.find('span', attrs={'class': 'status-il'}):
                    continue 
                
                player_name = unidecode.unidecode(row.a.get_text().strip())
                position_player_dict[position].append(player_name)
                
                if row.find('span', attrs={'class': 'closer'}):
                    position_player_dict['Closer'] = player_name 
                        
        team_roster_dict[nickname_to_city_dict[team_name]] = position_player_dict

        time.sleep(2)
        
        url = url_start+mlb_name+'/roster'

        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')

        # Getting player names on each roster
        for table in soup.findAll('table', attrs={'class': 'roster__table'}):
            position = table.td.get_text()

            if position=='Picthers':
                position = 'P'
            elif position=='Catchers':
                position = 'C'
            elif position=='Infielders':
                position = 'IF'
            elif position=='Outfielders':
                position = 'OF'
            elif position=='Designated Hitters':
                position = 'DH'

            position_player_dict[position] = []

            for row in table.tbody.findAll('td', attrs={'class': 'info'}):
                
                player_name = unidecode.unidecode(row.a.get_text().strip())
                position_player_dict[position].append(player_name)
        
                        
        team_roster_dict[nickname_to_city_dict[team_name]] = position_player_dict

        time.sleep(2)
                        
    return team_roster_dict

def closers():
    
    headers = {'User-Agent': header_name}

    url_list = []
    url_start = 'https://www.mlb.com/'
    
    team_closer_dict = {}
    
    for team_name in mlb_teams:
        
        mlb_name = None

        # Making sure all team names are in mlb.com format
        if team_name=='red sox':
            mlb_name = 'redsox'
        elif team_name=='blue jays':
            mlb_name = 'bluejays'
        elif team_name=='d-backs':
            mlb_name = 'dbacks'
        elif team_name=='white sox':
            mlb_name = 'whitesox'
        else:
            mlb_name = team_name
     
        url = url_start+mlb_name+'/roster/depth-chart'
        position_player_dict = {}

        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')

        # Getting player names on each depth chart
        for table in soup.findAll('table', attrs={'class': 'roster__table'}):
            position = table.td.get_text()

            if position=='Bullpen':
                position = 'RP'
            else:
                continue

            for row in table.tbody.findAll('td', attrs={'class': 'info'}):
                if row.find('span', attrs={'class': 'status-il'}):
                    continue 
                
                player_name = unidecode.unidecode(row.a.get_text().strip())
                
                if row.find('span', attrs={'class': 'closer'}):
                    position_player_dict['Closer'] = player_name 
                        
        team_closer_dict[nickname_to_city_dict[team_name]] = position_player_dict

        #time.sleep(1)
        
                        
    return team_closer_dict

def game_predictions(schedule_dict, team_dict):
    games = 500
    team_wins_dict = {}
    for matchups in schedule_dict.values():
        for matchup in matchups:
            if(len(lineup_dict[matchup[0]]['lineup'])<1 or len(lineup_dict[matchup[0]]['lineup'])<1):   # Only predict games with lineups for both teams
                continue
            for n in range(games):
                innings=9

                if(matchup[0][-1]=='2' or matchup[0]+'2' in team_dict.keys()):   # 7 inning double headers
                    innings = 7
                
                g = Game_new(team_dict[matchup[0]], team_dict[matchup[1]], innings)
                g.play()
                g.print_result()
            #print(matchup[0] + " win rate", team_dict[matchup[0]].wins/games)
            #print(matchup[0] +  " R/G", team_dict[matchup[0]].cum_runs/games)
            #print(matchup[1] + " wins rate", team_dict[matchup[1]].wins/games)
            #print(matchup[1] +  " R/G", team_dict[matchup[1]].cum_runs/games)
            Away_o, Away_p = team_dict[matchup[0]].cumul_stats_df()
            Home_o, Home_p = team_dict[matchup[1]].cumul_stats_df()
            #print(Away_o)
            #print(Away_p)
            #print(Home_o)
            #print(Home_p)
            #print(" ")
            #print(" ")
            #print(" ")
            #print(" ")
            #print(" ")
            #print(" ")
            #print(" ")

            team_wins_dict[matchup[0]] = round(team_dict[matchup[0]].wins/games, 2)
            team_wins_dict[matchup[1]] = round(team_dict[matchup[1]].wins/games, 2)

            team_dict[matchup[0]].restart(season=True)
            team_dict[matchup[1]].restart(season=True)

    return team_wins_dict

def moneylines_and_predictions(df, schedule_dict, team_dict, year):
    """Loads moneylines to spreadsheet
    Args:
        df: dataframe that odds will be written to
        scedule_dict: dict of games to be played (date as key, matchups as values)
        year: year that will be used to gather the dates of the games
    Returns:
        dataframe of data with moneylines
    """
    names = ['Open', 'ML','Westgate','MGM Mirage', 'betMGM',
             'William Hill', 'CG Technology', 'Circa Sports','Stations']
    book = pd.DataFrame(columns=names)
    temp_away_list = []
    temp_home_list = []
    teams_list = []
    dates_list = []
    URLs = ['http://www.vegasinsider.com/mlb/odds/las-vegas/', 'http://www.vegasinsider.com/mlb/odds/las-vegas/2']
    headers = {'User-Agent': header_name}

    for url in URLs:
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        for gameboard in soup.findAll('table', attrs={'class': 'viBodyContainerTble'}):
            for num, games in enumerate(gameboard.findAll('td', attrs={'class': 'viBodyBorderNorm'})):
                for matchup in games.findAll('tr'):
                    for game_info in matchup.findAll('td', attrs={'class': ['viCellBg1 cellTextNorm cellBorderL1 gameCell',
                                                                          'viCellBg2 cellTextNorm cellBorderL1 gameCell']}):

                        no_game = False

                        for date in game_info.findAll('span', attrs={'class': 'cellTextHot'}):   # Finding and formatting dates
                            date = date.get_text().split()[0]
                            month = date.split('/')[0]
                            day = date.split('/')[1]
                            date = str(month) + '/' + str(day) + '/' + str(year)
                            dates_list.extend([date, date])

                        for team_name in game_info.findAll('a', attrs={'class': 'tabletext'}): # Gathering team names
                            if(team_name.get_text().strip()=='N.Y. Yankees'):
                                teams_list.append('New York Yankees')
                            elif(team_name.get_text().strip()=='N.Y. Mets'):
                                teams_list.append('New York Mets')
                            elif(team_name.get_text().strip()=='Chi. Cubs'):
                                teams_list.append('Chicago Cubs')
                            elif(team_name.get_text().strip()=='Chi. White Sox'):
                                teams_list.append('Chicago White Sox')
                            elif(team_name.get_text().strip()=='L.A. Dodgers'):
                                teams_list.append('Los Angeles Dodgers')
                            elif(team_name.get_text().strip()=='L.A. Angels'):
                                teams_list.append('Los Angeles Angels')
                            else:
                                teams_list.append(team_name.get_text().strip())                  
                        
                        if(teams_list[-2] in teams_list[0:len(teams_list)-2]):   # Double headers
                            temp1 = teams_list[-2]
                            temp2 = teams_list[-2]
                            teams_list = teams_list[0:len(teams_list)-2]
                            teams_list.append(temp1+'2')
                            teams_list.append(temp2+'2')

                        if((teams_list[-2], teams_list[-1]) not in list(schedule_dict.values())[0]
                           or (len(lineup_dict[teams_list[-2]]['lineup'])<1 or len(lineup_dict[teams_list[-1]]['lineup'])<1)):   # Games not being played or without lineups
                            teams_list = teams_list[0:len(teams_list)-2]
                            dates_list = dates_list[0:len(dates_list)-2]
                            no_game = True

                        if(not no_game):
                            for num, spread in enumerate(matchup.findAll('td', attrs={'class': 
                                                                         ['viCellBg1 cellTextNorm cellBorderL1 center_text nowrap oddsCell',
                                                                          'viCellBg1 cellTextHot cellBorderL1 center_text nowrap oddsCell',
                                                                          'viCellBg2 cellTextNorm cellBorderL1 center_text nowrap oddsCell',
                                                                          'viCellBg2 cellTextHot cellBorderL1 center_text nowrap oddsCell']})):           
                                spread_text = spread.get_text().strip()

                                spread_away = str(spread).split('<br/>')[1]
                                if(len(spread_away.split())==0 or spread_away.split()[0]=='XX'):
                                    away_spread = ''
                                else:   
                                    away_spread = spread_away.split()[0]

                                spread_home = str(spread).split('<br/>')[2]
                                if(len(spread_home.split())<2 or spread_home.split()[0]=='XX'):
                                    home_spread = ''
                                else:   
                                    home_spread = spread_home.split()[0]


                                temp_away_list.append(away_spread)
                                temp_home_list.append(home_spread)

                                if len(temp_home_list) == 9:
                                    book = book.append(pd.Series(temp_away_list, index=names), ignore_index=True)
                                    book = book.append(pd.Series(temp_home_list, index=names), ignore_index=True)
                                    temp_away_list = []
                                    temp_home_list = []

    while(len(dates_list) > len(teams_list)):  # In case extra games in odds like All Star game
        dates_list.pop()
        book.drop(book.tail(1).index, inplace=True)

    book['Team'] = pd.Series(teams_list)
    
    book['Date'] = dates_list
    book['Version'] = '1.1'
    book = book[book.Date==datetime.datetime.now().strftime('%m/%d/%Y')]    # Only getting odds for games in current week
    
    team_wins_dict = game_predictions(schedule_dict, team_dict)
    
    book['Implied Probability'] = book['ML'].apply(lambda x: implied_probability(x))
    book['Jack Prediction'] = book['Team'].apply(lambda x: team_wins_dict[x])
    book['Jack Pick'] = book.apply(lambda x: ml_pick(x), axis=1)
    book['Winner'] = np.nan
    book['ML Result'] = np.nan
    book['Outcome'] = np.nan
    book['Total'] = np.nan
    print(book)
    odds = book[['Version', 'Date', 'Team', 'ML', 'Implied Probability', 'Jack Prediction', 'Jack Pick', 'Winner', 'ML Result', 'Outcome', 'Total']]
    #odds.set_index('Team', inplace=True)
    
    return odds

In [144]:
batter_stats = reading_data('Joined_Batter_Stats.csv')
batter_stats = batter_stats.replace(np.nan, 0)
pitcher_stats = reading_data('Joined_Pitcher_Stats.csv')
pitcher_stats = pitcher_stats.replace(np.nan, 0)
lineup_dict = {}

In [ ]:
#lineup_dict, schedule_dict, player_dict  = fangraphs_daily_lineups(lineup_dict)
team_dict, rosters = create_teams_new(batter_stats, pitcher_stats, lineup_dict, closers())
#new_games = loading_moneylines(pd.DataFrame(columns=['date', 'team', 'proj win%', 'ml']), schedule_dict, team_dict, year)

In [ ]:
odds_df = moneylines_and_predictions(pd.DataFrame(columns=['Date', 'Team', 'proj win%', 'ML']), schedule_dict, team_dict, 2021)
#odds_df.set_index(win_rates.index, inplace=True)
odds_df

In [156]:
def getting_spreadsheet(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    return(pd.read_csv(file_name))

def separating_games(df):
    """Separates old games and new games
    Args:
        df: dataframe of data
    Returns:
        old_games: games with results already
        current_game: games without results
    """
    old_games = df[df['Result'] != '']
    old_games_index = df[df['Result'] != ''].index 
    current_games = df.drop(old_games_index)
    current_games['Team'].replace('', np.nan, inplace=True)
    current_games.dropna(subset=['Team'], inplace=True)
    
    return old_games, current_games

def predictions_538(game_date):
    """Loads 538 predictions
    Args:
        game_date: date of games
    Returns:
        predictions: dataframe with 538 predictions
        year: year of games. Needed for Vegas odds
    """
    year = 2020
    result = ''
    URL = 'https://projects.fivethirtyeight.com/2021-mlb-predictions/games/'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    cols = ['Date', 'Team', '538 win%']
    blank_rows = ['']*(3)
    cols.extend(blank_rows)
    predictions = pd.DataFrame(columns=cols)

    # Gathering data
    for day_group in soup.findAll('div', attrs={'class': 'day-group'}):
        for day_of_year in day_group.findAll('section', attrs={'class': 'day'}):
            
            # Finding and formatting date of each game  
            date = day_of_year.h3.get_text().strip()
            date = date.split(', ')[1]
            m = date.split()[0]
            d = date.split()[1]
            date = m[0:3] + ' ' + d + ' ' + str(year)
            date = datetime.datetime.strptime(date, '%b %d %Y')
            year = date.year
            month = date.month
            day = date.day
            date = date.strftime('%m/%d/%Y')

            # Finding data for each game on a given date
            if date == game_date:
                for game in day_of_year.findAll('tbody', attrs={'class': 'ie10up'}):
                    for num_teams, data in enumerate(game.findAll('tr', attrs={'class': 'tr team'})): 
                        team_acronym = data['data-team']
                        squad = data.find('td', attrs={'class': 'td text team '+team_acronym})
                        win_percentage = data.find('td', attrs={'class': 'td number chance'}).get_text().strip()
                        spread = data.find('td', attrs={'class': 'td number spread'}).get_text().strip()

                        if squad:
                            team = squad.get_text().strip()
                            result = ''
                            
                            if num_teams == 0: # Gathering data for away team
                                team_away = team
                                win_percentage_away = win_percentage
                                spread_away = spread
                            else:
                                if spread_away == 'PK':
                                    spread_home = 'PK'
                                elif spread_away == '':
                                    spread_home = spread
                                    
                                    if spread_home == 'PK':
                                        spread_away = 'PK'
                                    else:
                                        spread_away = '+' + spread_home[1:]
                                else:
                                    spread_home = '+' + spread_away[1:]
                                
                                row_data_away = [date, team_away, win_percentage_away, spread_away]
                                row_data_away.extend(blank_rows)
                                row_data_home = [date, team, win_percentage, spread_home]
                                row_data_home.extend(blank_rows)
                                
                                predictions = predictions.append(pd.Series(row_data_away, index=predictions.columns), 
                                                                 ignore_index=True)
                                predictions = predictions.append(pd.Series(row_data_home, index=predictions.columns), 
                                                                 ignore_index=True)
            else:
                break
                        
                            
    return predictions, year   

def spreadsheet_date_formatter(date):
    """Formats dates
    Args:
        row: row of data
    Returns:
        date in proper form('mm/dd/yyyy')
    """
    if date == '':
        return(date)
    else:
        split_date = date.split('/')
        day = split_date[1]
        day = day.zfill(2)
        date = split_date[0] +  '/' + day + '/' + split_date[2]
        date = datetime.datetime.strptime(date, '%m/%d/%Y')
        date = date.strftime('%m/%d/%Y')

        return(date)
    
def date_ffill_formatter(df):
    """Formats dates to be in every other row
    Args:
        df: df of data
    """
    for index, row in df.iterrows():
        if index%2 == 0:
            continue
        else:
            df.at[index, 'Date'] = ''    
            df.at[index, 'Version'] = ''   
            
def combining_data(df_top, df_bottom):
    """Appends two dataframes
    Args:
        df_top: dataframe of data to go on top
        df_bottom: dataframe of data to go on bottom
    Returns:
        df: combined dataframe
    """
    cols = df_top.columns
    df = pd.concat([df_top, df_bottom], ignore_index=True, )
    df = df[cols]
    df.replace(np.nan, '', inplace=True)
    return(df)  

def game_outcomes(current_games, game_dates, teams):
    """Finds winners and losers of games
    Args:
        df: dataframe of data
        game_dates: dates of games completed
        teams: teams in games that are completed
    Returns:
        winners: dict with dates as keys and a list of winners as values
        losers: dict with dates as keys and a list of losers as values
    """
    year = 2021
    result = ''
    URL = 'https://projects.fivethirtyeight.com/2021-mlb-predictions/games/?ex_cid=rrpromo'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    winner_list = []
    loser_list = []
    winners = {}
    losers = {}
    #team_acronyms = [nickname_to_acronym_dict[team] for team in teams]
    results_df = pd.DataFrame(columns=['date', 'team', 'score', 'ml result'])

    
    team_scores = {}
    skip_game = False
    
    for games in soup.findAll('tbody'):
        for game in games.findAll('tr', attrs={'class': 'tr'}):
            
            if skip_game:
                skip_game = False
                continue
            
            if game.find('span', attrs={'class': 'day short'}):
                date = game.find('span', attrs={'class': 'day short'}).get_text()
                m = date.split('/')[0]
                d = date.split('/')[1]
                date = m + ' ' + d + ' ' + str(year)
                date = datetime.datetime.strptime(date, '%m %d %Y')
                year = date.year
                month = date.month
                day = date.day
                date = date.strftime('%m/%d/%Y')

                if date not in game_dates or date==datetime.datetime.now().strftime('%m/%d/%Y') :   # Skip games that don't need to be updated with scores
                    skip_game = True
                    continue
                
                away_team = nickname_to_city_dict[game.find('span', attrs={'class': 'team-name long'}).get_text().strip().lower()]
                away_row_index = current_games.index[(current_games['Date']==date) & (current_games['Team']==away_team)]

                if away_team not in teams:  # Skip games that don't need to be updated with scores
                    skip_game = True
                    continue

                away_score = game.find('td', attrs={'class': ['td number td-number score',
                                                              'td number td-number score is-winner']}).get_text().strip()
                current_games.at[away_row_index, 'Score'] = away_score

                if game.find('td', attrs={'class': 'td number td-number score is-winner'}):
                    current_games.at[away_row_index, 'Result'] = 'w'
                    current_games.at[away_row_index, 'Winner'] = away_team
                else:
                    current_games.at[away_row_index, 'Result'] = 'l'
                    current_games.at[away_row_index, 'Winner'] = ''
            else:
                home_team = nickname_to_city_dict[game.find('span', attrs={'class': 'team-name long'}).get_text().strip().lower()]
                home_row_index = current_games.index[(current_games['Date']==date) & (current_games['Team']==home_team)]

                home_score = game.find('td', attrs={'class': ['td number td-number score',
                                                              'td number td-number score is-winner']}).get_text().strip()
                current_games.at[home_row_index, 'Score'] = home_score
                
                if game.find('td', attrs={'class': 'td number td-number score is-winner'}):
                    current_games.at[home_row_index, 'Result'] = 'w'
                    current_games.at[home_row_index, 'Winner'] = home_team
                else:
                    current_games.at[home_row_index, 'Result'] = 'l'
                    current_games.at[home_row_index, 'Winner'] = ''
       
    """
    # Gathering data
    for day_group in soup.findAll('div', attrs={'class': 'completed-day'}):
        for day_of_year in day_group.findAll('section', attrs={'class': 'day'}):  # Finding and formatting dates
            date = day_of_year.h3.get_text().strip()
            date = date.split(', ')[1]
            m = date.split()[0]
            d = date.split()[1]
            date = m[0:3] + ' ' + d + ' ' + str(year)
            date = datetime.datetime.strptime(date, '%b %d %Y')
            year = date.year
            month = date.month
            day = date.day
            date = date.strftime('%m/%d/%Y')

            # Finding data for each game on a given date
            if date in game_dates:
                for team_acronym in team_acronyms:
                    for game in day_of_year.findAll('tbody', attrs={'class': 'ie10up'}):
                        for game_outcome in game.findAll('tr', attrs={'class': 'tr team'}): 
                            winner_string = 'td text team winner ' + team_acronym
                            loser_string = 'td text team loser ' + team_acronym

                            for winner in game_outcome.findAll('td', attrs={'class': winner_string}):
                                winner = winner.get_text().strip()
                                
                                for score in game_outcome.find('td', attrs={'class': 'td number score winner'}):
                                    index = df.loc[(df['date'] == date) & (df['team'] == winner)].index
                                    df.loc[index, 'score'] = score
                                    df.loc[index, 'ml result'] = 'w'
                                
                            for loser in game_outcome.findAll('td', attrs={'class': loser_string}):
                                loser = loser.get_text().strip()
                                
                                for score in game_outcome.find('td', attrs={'class': 'td number score loser'}):
                                    index = df.loc[(df['date'] == date) & (df['team'] == loser)].index
                                    df.loc[index, 'score'] = score
                                    df.loc[index, 'ml result'] = 'l'
                                
    index = df.loc[(df['ml result'] == 'w') | (df['ml result'] == 'l')].index
    if len(index) > 0:
        min_index = min(index)
        max_index = max(index)

        if (max_index-min_index)%2 ==1:
            for away_team_index in range(min_index, max_index, 2):
                if df.loc[away_team_index, 'spread'] == '':
                    continue
                else:
                    home_team_index = away_team_index + 1
                    away_score = float(df.loc[away_team_index, 'score'])
                    home_score = float(df.loc[home_team_index, 'score'])
                    away_spread = float(df.loc[away_team_index, 'spread'])
                    home_spread = float(df.loc[home_team_index, 'spread'])

                    if (away_score + away_spread) > home_score:
                        df.loc[away_team_index, 'spread result'] = 'w'
                        df.loc[home_team_index, 'spread result'] = 'l'
                    elif (away_score + away_spread) == home_score:
                        df.loc[away_team_index, 'spread result'] = 'push'
                        df.loc[home_team_index, 'spread result'] = 'push'
                    elif (away_score + away_spread) < home_score:
                        df.loc[away_team_index, 'spread result'] = 'l'
                        df.loc[home_team_index, 'spread result'] = 'w'
                    else:
                        continue
    """            
    return current_games 

def implied_probability(ML):
    """Uses odds to determine implied probability
    Args:
        row: row of data from dataframe
    Returns:
        implied win probability if it exists
    """

    if ML[0] == '+':
        return(round(100.0/(100+int(ML[1:])), 2))
    elif ML[0] == '-':
        return(round(int(ML[1:])/(100.0+int(ML[1:])), 2))
    else:
        return('')      
    
def ml_pick(row):
    """Uses win% and odds to determine what team to pick
    Args:
        row: row of data from dataframe
        predictor: name of predicting column
    Returns:
        pick if there is one
    """
    
    if row['Implied Probability'] == '':
        return('')
    elif (row['Jack Prediction'] > row['Implied Probability']):
        return(row.Team)
    else:
        return('')      
    
def spreadsheet_formatter(df):
    """Formats spreadsheet
    Args:
        df: dataframe of data
    """
    if 'Unnamed: 0' in df.columns:
        df.drop('Unnamed: 0', axis=1, inplace=True)
        
    df.replace(np.nan, '', inplace=True)  
    
def ml_formatter(row):
    """Formats ml
    Args:
        row: row of data from dataframe
    Returns:
        formatted odds
    """
    
    if row['ML'] == '':
        return(row['ML'])
    elif str(row['ML'])[0] == '+':
        return(str(row['ML']))
    elif int(row['ML']) >= 100:
        return('+' + str(int(row['ML'])))
    if int(row['ML']) <= -100:
        return('-' + str(int(abs(row['ML']))))
    else:
        return(str(row['ML']))
        
def team_won_lost(row, winners, losers):
    """Determines team outcomes
    Args:
        row: row of data from dataframe
        winners: dict with dates as keys and a list of winners as values
        losers: dict with dates as keys and a list of losers as values
    Returns:
        updated entry for winner/loser
    """
    if row.date != '':
        if row.date in winners.keys():
            if row.team in winners[row.date]:
                return('w')
            elif row.team in losers[row.date]:
                return('l')
            else:
                return('')
        else:
            return('')
    else:
        return('')   
    
def money_won_lost(row):
    """Determines amount won lost
    Args:
        row: row of data from dataframe
    Returns:
        amount won/lost
    """
    if(row['Jack Pick'] not in [None, '']):
        if row['Result'] == 'w':
            if row['ML'][0] == '+':
                return(int(row['ML'][1:]))
            else:
                return(100)
        elif row['Result'] == 'l':
            if row['ML'][0] == '+':
                return(-100)
            else:
                return(int(row['ML']))
        else:
            return(int(0))   
    else:
        return(int(0))

def writing_spreadsheet(df, filename):
    """Writing to spreadsheet
    Args:
        df: data
        file_name: path of file
    """
    df.to_csv(filename, index=False)

In [149]:
file_name = 'Baseball Picks.csv'
date = datetime.date.today().strftime('%m/%d/%Y')
year = 2021

df = getting_spreadsheet(file_name) # Retrieving Spreadsheet

spreadsheet_formatter(df)
df['ML'] = df.apply(lambda row: ml_formatter(row), axis=1)
old_games, current_games = separating_games(df)


if not current_games.empty:
    current_games['Date'] = current_games['Date'].apply(lambda date: spreadsheet_date_formatter(date))
    current_games['Date'] = current_games['Date'].replace('', np.nan).ffill()
    current_games = game_outcomes(current_games, current_games.Date.unique(), current_games.Team.unique())
    current_games['Outcome'] = current_games.apply(lambda row: money_won_lost(row), axis=1)
    df = combining_data(old_games, current_games)
    
df['Date'] = df['Date'].apply(lambda date: spreadsheet_date_formatter(date))
lineup_dict, schedule_dict, player_dict  = fangraphs_daily_lineups(lineup_dict)



if(date > df.Date.max() or len(df.loc[df['Date']==df.Date.max()])<len(schedule_dict[df.Date.max()])):
    lineup_dict, schedule_dict, player_dict  = fangraphs_daily_lineups(lineup_dict)
    team_dict, rosters = create_teams_new(batter_stats, pitcher_stats, lineup_dict, closers())
    new_games = moneylines_and_predictions(pd.DataFrame(columns=['date', 'team', 'proj win%', 'ml']), schedule_dict, team_dict, year)
    print(new_games)
        #new_games['implied'] = new_games.apply(lambda row: implied_probability(row), axis=1)
        #new_games['538 ml pick'] = new_games.apply(lambda row: ml_pick(row, '538'), axis=1)
        #new_games['538 ml w/l'] = 0
    df = combining_data(df, new_games)

df['Total'] = int(df.loc[(df['Outcome'] != '') & (df['Outcome'] != np.nan)]['Outcome'].sum())
date_ffill_formatter(df)
print(df)

#writing_spreadsheet(df, file_name)

HI
batter_name Tommy Pham
batter_name Fernando Tatis Jr.
batter_name Jake Cronenworth
batter_name Manny Machado
batter_name Eric Hosmer
batter_name Wil Myers
batter_name Trent Grisham
batter_name Victor Caratini
batter_name Joc Pederson
batter_name Dansby Swanson
batter_name Freddie Freeman
batter_name Ozzie Albies
batter_name Austin Riley
batter_name Stephen Vogt
batter_name Abraham Almonte
batter_name Guillermo Heredia
batter_name Brock Holt
batter_name Nate Lowe
batter_name Adolis Garcia
batter_name Joey Gallo
batter_name Jonah Heim
batter_name Eli White
batter_name David Dahl
batter_name Isiah Kiner-Falefa
batter_name Akil Baddoo
batter_name Jonathan Schoop
batter_name Robbie Grossman
batter_name Miguel Cabrera
batter_name Jeimer Candelario
batter_name Harold Castro
batter_name Victor Reyes
batter_name Alcides Escobar
batter_name Trea Turner
batter_name Juan Soto
batter_name Josh Bell
batter_name Josh Harrison
batter_name Andrew Stevenson
batter_name Gerardo Parra
batter_name Austi

final score: San Diego 4 Atlanta 5
final score: San Diego 3 Atlanta 2
final score: San Diego 3 Atlanta 4
final score: San Diego 11 Atlanta 4
final score: San Diego 0 Atlanta 7
final score: San Diego 5 Atlanta 2
final score: San Diego 7 Atlanta 2
final score: San Diego 2 Atlanta 5
final score: San Diego 3 Atlanta 7
final score: San Diego 0 Atlanta 1
final score: San Diego 2 Atlanta 7
final score: San Diego 4 Atlanta 9
final score: San Diego 2 Atlanta 6
final score: San Diego 8 Atlanta 14
final score: San Diego 5 Atlanta 6
final score: San Diego 0 Atlanta 3
final score: San Diego 4 Atlanta 1
final score: San Diego 8 Atlanta 7
final score: San Diego 13 Atlanta 5
final score: San Diego 5 Atlanta 6
final score: San Diego 3 Atlanta 2
final score: San Diego 1 Atlanta 10
final score: San Diego 9 Atlanta 2
final score: San Diego 17 Atlanta 9
final score: San Diego 7 Atlanta 5
final score: San Diego 2 Atlanta 5
final score: San Diego 6 Atlanta 5
final score: San Diego 8 Atlanta 0
final score: Sa

final score: San Diego 4 Atlanta 6
final score: San Diego 2 Atlanta 6
final score: San Diego 5 Atlanta 6
final score: San Diego 5 Atlanta 1
final score: San Diego 8 Atlanta 2
final score: San Diego 6 Atlanta 7
final score: San Diego 0 Atlanta 1
final score: San Diego 2 Atlanta 6
final score: San Diego 1 Atlanta 0
final score: San Diego 5 Atlanta 3
final score: San Diego 6 Atlanta 4
final score: San Diego 2 Atlanta 10
final score: San Diego 10 Atlanta 1
final score: San Diego 5 Atlanta 1
final score: San Diego 3 Atlanta 5
final score: San Diego 6 Atlanta 2
final score: San Diego 2 Atlanta 3
final score: San Diego 4 Atlanta 9
final score: San Diego 6 Atlanta 4
final score: San Diego 2 Atlanta 10
final score: San Diego 4 Atlanta 2
final score: San Diego 1 Atlanta 11
final score: San Diego 13 Atlanta 5
final score: San Diego 3 Atlanta 7
final score: San Diego 1 Atlanta 7
final score: San Diego 1 Atlanta 3
final score: San Diego 2 Atlanta 4
final score: San Diego 5 Atlanta 0
final score: Sa

final score: Texas 4 Detroit 9
final score: Texas 7 Detroit 1
final score: Texas 11 Detroit 8
final score: Texas 6 Detroit 9
final score: Texas 5 Detroit 2
final score: Texas 6 Detroit 8
final score: Texas 4 Detroit 2
final score: Texas 3 Detroit 4
final score: Texas 0 Detroit 2
final score: Texas 8 Detroit 7
final score: Texas 1 Detroit 7
final score: Texas 4 Detroit 9
final score: Texas 10 Detroit 4
final score: Texas 5 Detroit 6
final score: Texas 0 Detroit 3
final score: Texas 7 Detroit 4
final score: Texas 1 Detroit 3
final score: Texas 0 Detroit 6
final score: Texas 4 Detroit 1
final score: Texas 5 Detroit 0
final score: Texas 0 Detroit 4
final score: Texas 12 Detroit 2
final score: Texas 0 Detroit 2
final score: Texas 4 Detroit 9
final score: Texas 5 Detroit 8
final score: Texas 1 Detroit 0
final score: Texas 4 Detroit 6
final score: Texas 15 Detroit 14
final score: Texas 4 Detroit 5
final score: Texas 1 Detroit 3
final score: Texas 5 Detroit 10
final score: Texas 4 Detroit 2
fi

final score: Baltimore 6 Tampa Bay 3
final score: Baltimore 5 Tampa Bay 2
final score: Baltimore 4 Tampa Bay 3
final score: Baltimore 3 Tampa Bay 13
final score: Baltimore 7 Tampa Bay 3
final score: Baltimore 4 Tampa Bay 0
final score: Baltimore 5 Tampa Bay 7
final score: Baltimore 2 Tampa Bay 3
final score: Baltimore 5 Tampa Bay 4
final score: Baltimore 4 Tampa Bay 7
final score: Baltimore 4 Tampa Bay 3
final score: Baltimore 6 Tampa Bay 3
final score: Baltimore 6 Tampa Bay 4
final score: Baltimore 0 Tampa Bay 2
final score: Baltimore 2 Tampa Bay 3
final score: Baltimore 2 Tampa Bay 5
final score: Baltimore 5 Tampa Bay 7
final score: Baltimore 6 Tampa Bay 2
final score: Baltimore 3 Tampa Bay 6
final score: Baltimore 6 Tampa Bay 4
final score: Baltimore 3 Tampa Bay 12
final score: Baltimore 0 Tampa Bay 9
final score: Baltimore 1 Tampa Bay 4
final score: Baltimore 6 Tampa Bay 3
final score: Baltimore 2 Tampa Bay 4
final score: Baltimore 4 Tampa Bay 8
final score: Baltimore 2 Tampa Bay 5

...

final score: Pittsburgh 5 Arizona 3
final score: Pittsburgh 8 Arizona 6
final score: Pittsburgh 2 Arizona 3
final score: Pittsburgh 0 Arizona 3
final score: Pittsburgh 10 Arizona 2
final score: Pittsburgh 6 Arizona 4
final score: Pittsburgh 5 Arizona 8
final score: Pittsburgh 2 Arizona 3
final score: Pittsburgh 7 Arizona 4
final score: Pittsburgh 5 Arizona 3
final score: Pittsburgh 2 Arizona 0
final score: Pittsburgh 4 Arizona 1
final score: Pittsburgh 1 Arizona 3
final score: Pittsburgh 3 Arizona 11
final score: Pittsburgh 7 Arizona 9
final score: Pittsburgh 7 Arizona 3
final score: Pittsburgh 2 Arizona 1
final score: Pittsburgh 8 Arizona 7
final score: Pittsburgh 4 Arizona 3
final score: Pittsburgh 4 Arizona 3
final score: Pittsburgh 6 Arizona 8
final score: Pittsburgh 0 Arizona 3
final score: Pittsburgh 13 Arizona 4
final score: Pittsburgh 13 Arizona 5
final score: Pittsburgh 3 Arizona 6
final score: Pittsburgh 1 Arizona 4
final score: Pittsburgh 2 Arizona 1
final score: Pittsburgh 

final score: Pittsburgh 6 Arizona 2
final score: Pittsburgh 11 Arizona 6
final score: Pittsburgh 7 Arizona 5
final score: Pittsburgh 3 Arizona 8
final score: Pittsburgh 6 Arizona 3
final score: Pittsburgh 5 Arizona 4
final score: Pittsburgh 0 Arizona 7
final score: Pittsburgh 7 Arizona 4
final score: Pittsburgh 5 Arizona 4
final score: Pittsburgh 4 Arizona 8
final score: Pittsburgh 2 Arizona 5
final score: Pittsburgh 7 Arizona 4
final score: Pittsburgh 0 Arizona 1
final score: Pittsburgh 1 Arizona 7
final score: Pittsburgh 1 Arizona 4
final score: Pittsburgh 2 Arizona 7
final score: Pittsburgh 6 Arizona 1
final score: Pittsburgh 4 Arizona 0
final score: Pittsburgh 1 Arizona 5
final score: Pittsburgh 2 Arizona 6
final score: Pittsburgh 7 Arizona 3
final score: Pittsburgh 3 Arizona 5
final score: Pittsburgh 0 Arizona 2
final score: Pittsburgh 6 Arizona 4
final score: Pittsburgh 3 Arizona 14
final score: Pittsburgh 3 Arizona 2
final score: Pittsburgh 5 Arizona 6
final score: Pittsburgh 2 

final score: Los Angeles Angels 8 Oakland 0
final score: Los Angeles Angels 5 Oakland 0
final score: Los Angeles Angels 4 Oakland 0
final score: Los Angeles Angels 12 Oakland 5
final score: Los Angeles Angels 5 Oakland 4
final score: Los Angeles Angels 2 Oakland 0
final score: Los Angeles Angels 1 Oakland 9
final score: Los Angeles Angels 2 Oakland 0
final score: Los Angeles Angels 6 Oakland 4
final score: Los Angeles Angels 4 Oakland 0
final score: Los Angeles Angels 9 Oakland 5
final score: Los Angeles Angels 6 Oakland 4
final score: Los Angeles Angels 1 Oakland 2
final score: Los Angeles Angels 3 Oakland 1
final score: Los Angeles Angels 1 Oakland 0
final score: Los Angeles Angels 3 Oakland 0
final score: Los Angeles Angels 3 Oakland 2
final score: Los Angeles Angels 2 Oakland 7
final score: Los Angeles Angels 4 Oakland 5
final score: Los Angeles Angels 2 Oakland 6
final score: Los Angeles Angels 15 Oakland 1
final score: Los Angeles Angels 5 Oakland 1
final score: Los Angeles Angel

final score: Los Angeles Angels 1 Oakland 3
final score: Los Angeles Angels 3 Oakland 4
final score: Los Angeles Angels 5 Oakland 7
final score: Los Angeles Angels 4 Oakland 3
final score: Los Angeles Angels 5 Oakland 0
final score: Los Angeles Angels 7 Oakland 8
final score: Los Angeles Angels 1 Oakland 5
final score: Los Angeles Angels 1 Oakland 3
final score: Los Angeles Angels 2 Oakland 3
final score: Los Angeles Angels 1 Oakland 0
final score: Los Angeles Angels 10 Oakland 2
final score: Los Angeles Angels 2 Oakland 4
final score: Los Angeles Angels 1 Oakland 3
final score: Los Angeles Angels 4 Oakland 12
final score: Los Angeles Angels 5 Oakland 2
final score: Los Angeles Angels 4 Oakland 0
final score: Los Angeles Angels 1 Oakland 0
final score: Los Angeles Angels 7 Oakland 5
final score: Los Angeles Angels 4 Oakland 7
final score: Los Angeles Angels 3 Oakland 4
final score: Los Angeles Angels 3 Oakland 2
final score: Los Angeles Angels 4 Oakland 0
final score: Los Angeles Angel

<ipython-input-119-5ba234adda8d>:146: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat([df_top, df_bottom], ignore_index=True, )


In [146]:
schedule_dict

{'2021-07-19': [('San Diego', 'Atlanta'),
  ('Texas', 'Detroit'),
  ('Miami', 'Washington'),
  ('Baltimore', 'Tampa Bay'),
  ('New York Mets', 'Cincinnati'),
  ('Boston', 'Toronto'),
  ('Chicago Cubs', 'St. Louis'),
  ('Cleveland', 'Houston'),
  ('Pittsburgh', 'Arizona'),
  ('Minnesota', 'Chicago White Sox'),
  ('San Francisco', 'Los Angeles Dodgers'),
  ('Los Angeles Angels', 'Oakland')]}

In [157]:
df

,Version,Date,Team,ML,Implied Odds,Jack Prediction,Jack Pick,Score,Winner,Result,Outcome,Total
0,1.1,07/16/2021,Minnesota,-160,,0.59,,,,ppd,,-649
1,,,Detroit,+150,,0.41,Tigers,,,ppd,,-649
2,1.1,07/16/2021,Houston,-120,,0.64,Astros,,Astros,w,100,-649
3,,,Chicago White Sox,+110,,0.36,,,,l,0,-649
4,1.1,07/16/2021,Baltimore,+140,,0.50,Orioles,,,l,-100,-649
5,,,Kansas City,-150,,0.50,,,Royals,w,0,-649
6,1.1,07/16/2021,Seattle,+133,,0.45,Mariners,,Mariners,w,133,-649
7,,,Los Angeles Angels,-143,,0.55,,,,l,0,-649
8,1.1,07/16/2021,Los Angeles Dodgers,-240,,0.67,,,Dodgers,w,0,-649
9,,,Colorado,+210,,0.33,Rockies,,,l,-100,-649


In [158]:
writing_spreadsheet(df, file_name)

In [ ]:
batter_stats = column_type(reading_data('Joined_Batter_Stats.csv'), True)
pitcher_stats = column_type(reading_data('Joined_Pitcher_Stats.csv'), True)
lineup_dict = None
lineup_dict, schedule_dict, player_dict  = fangraphs_daily_lineups(lineup_dict)
#rosters = active_rosters()
team_dict, rosters = create_teams_new(batter_stats, pitcher_stats, lineup_dict, closers())

#print(lineup_dict)
#print(schedule_dict)
#print(player_dict)
#print(rosters)
#print(team_dict)